<a href="https://colab.research.google.com/github/Onr/Council-GAN/blob/master/Council_gan_simple_infrence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installes



In [1]:
!pip install torchfile 

  Created wheel for torchfile: filename=torchfile-0.1.0-cp36-none-any.whl size=5712 sha256=8076650ea85ba2dddb281597737656e7215dc94bd16182880fa30351febd1c4f
  Stored in directory: /root/.cache/pip/wheels/b1/c3/d6/9a1cc8f3a99a0fc1124cae20153f36af59a6e683daca0a0814
Successfully built torchfile


#data.py


In [2]:
"""
Copyright (C) 2018 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""
import torch.utils.data as data
import os.path
import torch
import numpy as np

def default_loader(path):
    return Image.open(path).convert('RGB')


def default_flist_reader(flist):
    """
    flist format: impath label\nimpath label\n ...(same to caffe's filelist)
    """
    imlist = []
    with open(flist, 'r') as rf:
        for line in rf.readlines():
            impath = line.strip()
            imlist.append(impath)

    return imlist


class ImageFilelist(data.Dataset):
    def __init__(self, root, flist, transform=None,
                 flist_reader=default_flist_reader, loader=default_loader):
        self.root = root
        self.imlist = flist_reader(flist)
        self.transform = transform
        self.loader = loader

    def __getitem__(self, index):
        impath = self.imlist[index]
        img = self.loader(os.path.join(self.root, impath))
        if self.transform is not None:
            img = self.transform(img)

        return img

    def __len__(self):
        return len(self.imlist)


class ImageLabelFilelist(data.Dataset):
    def __init__(self, root, flist, transform=None,
                 flist_reader=default_flist_reader, loader=default_loader):
        self.root = root
        self.imlist = flist_reader(os.path.join(self.root, flist))
        self.transform = transform
        self.loader = loader
        self.classes = sorted(list(set([path.split('/')[0] for path in self.imlist])))
        self.class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}
        self.imgs = [(impath, self.class_to_idx[impath.split('/')[0]]) for impath in self.imlist]

    def __getitem__(self, index):
        impath, label = self.imgs[index]
        img = self.loader(os.path.join(self.root, impath))
        if self.transform is not None:
            img = self.transform(img)
        return img, label

    def __len__(self):
        return len(self.imgs)

###############################################################################
# Code from
# https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
# Modified the original code so that it also loads images from the current
# directory as well as the subdirectories
###############################################################################

import torch.utils.data as data

from PIL import Image
import os
import os.path

IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]


def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)


def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)
            elif fname.endswith('.npy'):
                path = os.path.join(root, fname)
                images.append(path)

    return images


class ImageFolder(data.Dataset):

    def __init__(self, root, transform=None, return_paths=False,
                 loader=default_loader):
        imgs = sorted(make_dataset(root))
        if len(imgs) == 0:
            raise(RuntimeError("Found 0 images in: " + root + "\n"
                               "Supported image extensions are: " +
                               ",".join(IMG_EXTENSIONS)))

        self.root = root
        self.imgs = imgs
        self.transform = transform
        self.return_paths = return_paths
        self.loader = loader

        # TODO tmp
        self.mean = 0
        self.std = None
        self.mean_pow2 = 0

        self.num_of_sample = 0


    def __getitem__(self, index):
        path = self.imgs[index]
        if not path.endswith('.npy'):
            try:
                img = self.loader(path)
            except Exception as e:
                print(str(e))
                del self.imgs[index]
                self.__getitem__(self, index)
            if self.transform is not None:
                img = self.transform(img)
        else:  # numpy data input # for brats dataset  # TODO add transforms
            img = torch.from_numpy(np.load(path))
            img = img.transpose(dim0=2, dim1=0)
            img = img.transpose(dim0=1, dim1=2)
            img = img.to(dtype=torch.float)
            from torchvision import transforms
            import torch.nn.functional as F
            mean_vec = torch.mean(img, dim=(1, 2))
            std_vec = torch.std(img, dim=(1, 2))
            img = transforms.Normalize(mean=mean_vec, std=std_vec)(img)
            size = [elem for elem in self.transform.transforms if type(elem) == transforms.transforms.Resize][0].size
            img = F.interpolate(input=torch.unsqueeze(img,0), size=size)
            img = torch.squeeze(img)
            # img = img[:-1,:,:]

            # dim0 Flair
            # dim1 T1
            # dim2 T1ce
            # dim3 T2
            # dim4 Seg

        if self.return_paths:
            return img, path
        else:
            return img

    def __len__(self):
        return len(self.imgs)




class ImageFolder_with_subfolders(data.Dataset):
# choose the data from the subfolder acording to probability
    # TODO complete usege
    def __init__(self, root1, root2, ratio_1_to_2, transform=None, return_paths=False,
                 loader=default_loader):
        self.ratio_1_to_2 = ratio_1_to_2
        imgs1 = sorted(make_dataset(root1))
        imgs2 = sorted(make_dataset(root2))
        if len(imgs1) == 0:
            raise(RuntimeError("Found 0 images in: " + root1 + "\n"
                               "Supported image extensions are: " +
                               ",".join(IMG_EXTENSIONS)))
        if len(imgs2) == 0:
            raise(RuntimeError("Found 0 images in: " + root2 + "\n"
                               "Supported image extensions are: " +
                               ",".join(IMG_EXTENSIONS)))

        self.root1 = root1
        self.root2 = root2
        self.imgs1 = imgs1
        self.imgs2 = imgs2
        self.transform = transform
        self.return_paths = return_paths
        self.loader = loader

    def __getitem__(self, index):
        rand_res = torch.rand(1)
        path = self.imgs1[index % len(self.imgs1)] if rand_res < self.ratio_1_to_2 else self.imgs2[index % len(self.imgs2)]
        img = self.loader(path)
        if self.transform is not None:
            img = self.transform(img)
        if self.return_paths:
            return img, path
        else:
            return img

    def __len__(self):
        return max(len(self.imgs1), len(self.imgs2))


# networks.py

In [3]:
"""
Copyright (C) 2018 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""
from torch import nn
from torch.autograd import Variable
import torch
import torch.nn.functional as F
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass

##################################################################################
# Discriminator
##################################################################################
class MsImageDis(nn.Module):
    # Multi-scale discriminator architecture
    def __init__(self, input_dim, params, cuda_device='cuda:0'):
        super(MsImageDis, self).__init__()
        self.prev_real_input = None
        self.n_layer = params['n_layer']
        self.gan_type = params['gan_type']
        self.dim = params['dim']
        self.norm = params['norm']
        self.activ = params['activ']
        self.num_scales = params['num_scales']
        self.pad_type = params['pad_type']
        self.cuda_device = cuda_device

        self.input_dim = input_dim
        self.downsample = nn.AvgPool2d(3, stride=2, padding=[1, 1], count_include_pad=False)
        self.cnns = nn.ModuleList()
        for _ in range(self.num_scales):
            self.cnns.append(self._make_net().cuda(self.cuda_device))

    def _make_net(self):
        dim = self.dim
        cnn_x = []
        cnn_x += [Conv2dBlock(self.input_dim, dim, 4, 2, 1, norm='none', activation=self.activ, pad_type=self.pad_type)]
        for i in range(self.n_layer - 1):
            cnn_x += [Conv2dBlock(dim, dim * 2, 4, 2, 1, norm=self.norm, activation=self.activ, pad_type=self.pad_type)]
            dim *= 2
        cnn_x += [nn.Conv2d(dim, 1, 1, 1, 0)]
        cnn_x = nn.Sequential(*cnn_x)
        return cnn_x

    def forward(self, x):
        outputs = []
        x = x.cuda(self.cuda_device)
        for model in self.cnns:
            outputs.append(model(x))
            x = self.downsample(x)
        return outputs

    def calc_dis_loss(self, input_fake, input_real):
        # calculate the loss to train D
        outs0 = self.forward(input_fake)
        outs1 = self.forward(input_real)
        loss = 0

        for it, (out0, out1) in enumerate(zip(outs0, outs1)):
            if self.gan_type == 'lsgan':
                loss += torch.mean((out0 - 0)**2) + torch.mean((out1 - 1)**2)
            elif self.gan_type == 'nsgan':
                all0 = Variable(torch.zeros_like(out0.data).cuda(self.cuda_device), requires_grad=False)
                all1 = Variable(torch.ones_like(out1.data).cuda(self.cuda_device), requires_grad=False)
                loss += torch.mean(F.binary_cross_entropy(F.sigmoid(out0), all0) +
                                   F.binary_cross_entropy(F.sigmoid(out1), all1))
            elif self.gan_type == 'RelativisticAverageHingeGAN':
                self.prev_real_input = input_real  # save it for the gen train later
                # difference between real and fake:
                r_f_diff = out1 - torch.mean(out0, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                # difference between fake and real samples
                f_r_diff = out0 - torch.mean(out1, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                # return the loss
                loss += (torch.mean(torch.nn.ReLU()(1 - r_f_diff))
                         + torch.mean(torch.nn.ReLU()(1 + f_r_diff)))

            else:
                assert 0, "Unsupported GAN type: {}".format(self.gan_type)
        return loss

    def calc_gen_loss(self, input_fake, input_real=None):
        # calculate the loss to train G
        outs0 = self.forward(input_fake)
        loss = 0
        for it, (out0) in enumerate(outs0):
            if self.gan_type == 'lsgan':
                loss += torch.mean((out0 - 1)**2)  # LSGAN
            elif self.gan_type == 'nsgan':
                all1 = Variable(torch.ones_like(out0.data).cuda(self.cuda_device), requires_grad=False)
                loss += torch.mean(F.binary_cross_entropy(F.sigmoid(out0), all1))
            elif self.gan_type == 'RelativisticAverageHingeGAN':
                if input_real is not None:
                    outs1 = self.forward(input_real)
                elif self.prev_real_input is not None:
                    outs1 = self.forward(self.prev_real_input)
                else:
                    assert 0, "try using cal_gan_loss with RelativisticAverageHingeGAN but did not provid input_real"
                out1 = outs1[it]
                # difference between real and fake:
                r_f_diff = out1 - torch.mean(out0, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                # difference between fake and real samples
                f_r_diff = out0 - torch.mean(out1, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                loss = torch.mean(torch.nn.ReLU()(1 + r_f_diff)) + torch.mean(torch.nn.ReLU()(1 - f_r_diff))

            else:
                assert 0, "Unsupported GAN type: {}".format(self.gan_type)
        return loss


##################################################################################
# Council Discriminator
##################################################################################
class MsImageDisCouncil(nn.Module):
    # Multi-scale discriminator architecture
    def __init__(self, input_dim, params, cuda_device='cuda:0'):
        super(MsImageDisCouncil, self).__init__()
        self.n_layer = params['n_layer']
        self.gan_type = params['gan_type']
        self.dim = params['dim']
        self.norm = params['norm']
        self.activ = params['activ']
        self.num_scales = params['num_scales']
        self.pad_type = params['pad_type']
        self.cuda_device = cuda_device
        self.input_dim = input_dim
        self.downsample = nn.AvgPool2d(3, stride=2, padding=[1, 1], count_include_pad=False)
        self.cnns = nn.ModuleList()
        for _ in range(self.num_scales):
            self.cnns.append(self._make_net())

    def _make_net(self):
        dim = self.dim
        cnn_x = []
        # cnn_x += [Conv2dBlock(input_dim=2 * self.input_dim, output_dim=dim, kernel_size=4, stride=2, padding=1, norm='none', activation=self.activ, pad_type=self.pad_type)] # original
        cnn_x += [Conv2dBlock(input_dim=2 * self.input_dim, output_dim=dim, kernel_size=3, stride=1, padding=1, norm='none', activation=self.activ, pad_type=self.pad_type)] # ON
        for i in range(self.n_layer - 1):
            cnn_x += [Conv2dBlock(dim, dim * 2, 4, 2, 1, norm=self.norm, activation=self.activ, pad_type=self.pad_type)]
            dim *= 2
        cnn_x += [nn.Conv2d(dim, dim, 1, 1, 0)]
        cnn_x += [nn.Conv2d(dim, 1, 1, 1, 0)]
        cnn_x = nn.Sequential(*cnn_x).cuda(self.cuda_device)
        return cnn_x

    def forward(self, x, x_input):
        x = x.cuda(self.cuda_device)
        x_input = x_input.cuda(self.cuda_device)
        outputs = []
        for model in self.cnns:
            model_input = torch.cat((x, x_input), 1)
            outputs.append(model(model_input))
            x = self.downsample(x)
            x_input = self.downsample(x_input)
        return outputs

    def calc_dis_loss(self, input_fake, input_real, input):
        # calculate the loss to train D
        outs0 = self.forward(input_fake, input)
        outs1 = self.forward(input_real, input)
        loss = 0

        for it, (out0, out1) in enumerate(zip(outs0, outs1)):
            if self.gan_type == 'lsgan':
                loss += torch.mean((out0 - 0)**2) + torch.mean((out1 - 1)**2)
            elif self.gan_type == 'nsgan':
                all0 = Variable(torch.zeros_like(out0.data).cuda(self.cuda_device), requires_grad=False)
                all1 = Variable(torch.ones_like(out1.data).cuda(self.cuda_device), requires_grad=False)
                loss += torch.mean(F.binary_cross_entropy(F.sigmoid(out0), all0) +
                                   F.binary_cross_entropy(F.sigmoid(out1), all1))

            elif self.gan_type == 'RelativisticAverageHingeGAN':
                self.prev_real_input = input_real  # save it for the gen train later
                self.prev_input = input  # save it for the gen train later
                # difference between real and fake:
                r_f_diff = out1 - torch.mean(out0, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                # difference between fake and real samples
                f_r_diff = out0 - torch.mean(out1, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                loss = torch.mean(torch.nn.ReLU()(1 + r_f_diff)) + torch.mean(torch.nn.ReLU()(1 - f_r_diff))
                # return the loss
                loss += (torch.mean(torch.nn.ReLU()(1 - r_f_diff))
                         + torch.mean(torch.nn.ReLU()(1 + f_r_diff)))
            else:
                assert 0, "Unsupported GAN type: {}".format(self.gan_type)
        return loss

    def calc_gen_loss(self, input_fake, input, input_real=None):
        # calculate the loss to train G
        outs0 = self.forward(input_fake, input)
        loss = 0
        for it, (out0) in enumerate(outs0):
            if self.gan_type == 'lsgan':
                loss += torch.mean((out0 - 1)**2) # LSGAN
            elif self.gan_type == 'nsgan':
                all1 = Variable(torch.ones_like(out0.data).cuda(self.cuda_device), requires_grad=False)
                loss += torch.mean(F.binary_cross_entropy(F.sigmoid(out0), all1))
            elif self.gan_type == 'RelativisticAverageHingeGAN':
                if input_real is not None:
                    outs1 = self.forward(input_real)
                elif self.prev_real_input is not None:
                    outs1 = self.forward(self.prev_real_input, self.prev_input)
                else:
                    assert 0, "try using cal_gan_loss with RelativisticAverageHingeGAN but did not provid input_real"

                out1 = outs1[it]
                # difference between real and fake:
                r_f_diff = out1 - torch.mean(out0, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                # difference between fake and real samples
                f_r_diff = out0 - torch.mean(out1, dim=0, keepdim=True).repeat(10, 1, 1, 1)
                loss = torch.mean(torch.nn.ReLU()(1 + r_f_diff)) + torch.mean(torch.nn.ReLU()(1 - f_r_diff))
                loss = torch.mean(torch.nn.ReLU()(1 + r_f_diff)) + torch.mean(torch.nn.ReLU()(1 - f_r_diff))
            else:
                assert 0, "Unsupported GAN type: {}".format(self.gan_type)
        return loss



##################################################################################
# Generator
##################################################################################

class AdaINGen(nn.Module):
    # AdaIN auto-encoder architecture
    def __init__(self, input_dim, params, cuda_device='cuda:0'):
        super(AdaINGen, self).__init__()
        dim = params['dim']
        style_dim = params['style_dim']
        self.n_downsample = params['n_downsample']
        n_res = params['n_res']
        self.activ = params['activ']
        pad_type = params['pad_type']
        mlp_dim = params['mlp_dim']
        self.do_my_style = params['do_my_style']
        self.cuda_device = cuda_device

        # style encoder
        self.enc_style = StyleEncoder(4, input_dim, dim, style_dim, norm='none', activ=self.activ, pad_type=pad_type).cuda(self.cuda_device)

        # content encoder
        self.enc_content = ContentEncoder(self.n_downsample, n_res, input_dim, dim, 'in', self.activ, pad_type=pad_type).cuda(self.cuda_device)

        if self.do_my_style:
            self.dec = Decoder_V2_atten(n_upsample=self.n_downsample, n_res=n_res, dim=self.enc_content.output_dim + style_dim, output_dim=input_dim, res_norm='in', activ=self.activ, pad_type=pad_type, num_of_mask_dim_to_add=params['num_of_mask_dim_to_add']).cuda(self.cuda_device)
        else:
            self.dec = Decoder_V2_atten(self.n_downsample, n_res, self.enc_content.output_dim, input_dim, res_norm='adain', activ=self.activ, pad_type=pad_type, num_of_mask_dim_to_add=params['num_of_mask_dim_to_add']).cuda(self.cuda_device)

        # MLP to generate AdaIN parameters or adding sytle my way
        if self.do_my_style:
            self.mlp = MLP(input_dim=style_dim, output_dim=style_dim, dim=mlp_dim, n_blk=3, norm='none',
                           activ=self.activ).cuda(self.cuda_device)
        else:
            self.mlp = MLP(input_dim=style_dim, output_dim=self.get_num_adain_params(self.dec), dim=mlp_dim, n_blk=3,
                           norm='none', activ=self.activ).cuda(self.cuda_device)

    def forward(self, images, return_mask=False):
        # reconstruct an image
        images = images.cuda(self.cuda_device)
        content, style_fake = self.encode(images)
        if return_mask:
            images_recon, mask = self.decode(content=content, style=style_fake, images=images, return_mask=return_mask)
            return images_recon, mask
        else:
            images_recon = self.decode(content=content, style=style_fake, images=images, return_mask=return_mask)
            return images_recon

    def forward(self, images, style, return_mask=False):
        # reconstruct an image
        content, _ = self.encode(images)
        if return_mask:
            images_recon, mask = self.decode(content=content, style=style, images=images, return_mask=return_mask)
            return images_recon, mask
        else:
            images_recon = self.decode(content=content, style=style, images=images, return_mask=return_mask)
            return images_recon


    def encode(self, images):
        images = images.cuda(self.cuda_device)
        # encode an image to its content and style codes
        style_fake = self.enc_style(images)
        content = self.enc_content(images)
        return content, style_fake

    def decode(self, content, style, images, return_mask=False):
        content, style, images = content.cuda(self.cuda_device), style.cuda(self.cuda_device), images.cuda(self.cuda_device)
        # decode content and style codes to an image
        if self.do_my_style:
            style_to_add = self.mlp(style)
            style_to_add = style_to_add.repeat(content.shape[2], content.shape[3], 1, 1)
            style_to_add = style_to_add.transpose(0, 2).transpose(3, 1)
            content = torch.cat((content, style_to_add), 1)
        else:
            adain_params = self.mlp(style)
            self.assign_adain_params(adain_params, self.dec)
        if return_mask:
            images, mask = self.dec(content, images, return_mask)
            return images, mask
        else:
            images = self.dec(content, images)
            return images

    def assign_adain_params(self, adain_params, model):
        # assign the adain_params to the AdaIN layers in model
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                mean = adain_params[:, :m.num_features]
                std = adain_params[:, m.num_features:2*m.num_features]
                m.bias = mean.contiguous().view(-1)
                m.weight = std.contiguous().view(-1)
                if adain_params.size(1) > 2*m.num_features:
                    adain_params = adain_params[:, 2*m.num_features:]

    def get_num_adain_params(self, model):
        # return the number of AdaIN parameters needed by the model
        num_adain_params = 0
        for m in model.modules():
            if m.__class__.__name__ == "AdaptiveInstanceNorm2d":
                num_adain_params += 2*m.num_features
        return num_adain_params


##################################################################################
# Encoder and Decoders
##################################################################################

class StyleEncoder(nn.Module):
    def __init__(self, n_downsample, input_dim, dim, style_dim, norm, activ, pad_type):
        super(StyleEncoder, self).__init__()
        self.model = []
        self.model += [Conv2dBlock(input_dim, dim, 7, 1, 3, norm=norm, activation=activ, pad_type=pad_type)]
        for i in range(2):
            self.model += [Conv2dBlock(dim, 2 * dim, 4, 2, 1, norm=norm, activation=activ, pad_type=pad_type)]
            dim *= 2
        for i in range(n_downsample - 2):
            self.model += [Conv2dBlock(dim, dim, 4, 2, 1, norm=norm, activation=activ, pad_type=pad_type)]
        self.model += [nn.AdaptiveAvgPool2d(1)] # global average pooling
        self.model += [nn.Conv2d(dim, style_dim, 1, 1, 0)]
        self.model = nn.Sequential(*self.model)
        self.output_dim = dim

    def forward(self, x):
        return self.model(x)

class ContentEncoder(nn.Module):
    def __init__(self, n_downsample, n_res, input_dim, dim, norm, activ, pad_type):
        super(ContentEncoder, self).__init__()
        self.model = []
        self.model += [Conv2dBlock(input_dim, dim, 7, 1, 3, norm=norm, activation=activ, pad_type=pad_type)] # ORIGINAL
        for i in range(n_downsample):
            self.model += [Conv2dBlock(dim, 2 * dim, 4, 2, 1, norm=norm, activation=activ, pad_type=pad_type)]
            dim *= 2
        # residual blocks
        self.model += [ResBlocks(n_res, dim, norm=norm, activation=activ, pad_type=pad_type)]
        self.model = nn.Sequential(*self.model)
        self.output_dim = dim

    def forward(self, x):
        return self.model(x)




class Decoder_V2_atten(nn.Module):
    def __init__(self, n_upsample, n_res, dim, output_dim, res_norm='adain', activ='relu', pad_type='zero', num_of_mask_dim_to_add=1):
        super(Decoder_V2_atten, self).__init__()
        self.num_of_mask_dim_to_add = num_of_mask_dim_to_add # 3  # 2
        self.model = []
        self.output_dim = output_dim
        self.mask_s = []
        # AdaIN residual blocks
        self.model += [ResBlocks(n_res, dim, res_norm, activ, pad_type=pad_type)]
        # upsampling blocks
        for i in range(n_upsample):
            self.model += [nn.Upsample(scale_factor=2)]
            self.model += [Conv2dBlock(input_dim=dim, output_dim=dim // 2, kernel_size=3, stride=1, padding=1, norm='adain',
                                       activation=activ, pad_type=pad_type)]
            dim //= 2
            self.model += [Conv2dBlock(input_dim=dim, output_dim=dim, kernel_size=3, stride=1, padding=1, norm='adain',
                                       activation=activ, pad_type=pad_type)]


        self.model += [Conv2dBlock(input_dim=dim, output_dim=dim, kernel_size=1, stride=1, padding=0, norm='none', activation=activ, pad_type=pad_type)]
        self.model += [Conv2dBlock(input_dim=dim, output_dim=dim, kernel_size=1, stride=1, padding=0, norm='none', activation=activ, pad_type=pad_type)]
        self.model += [Conv2dBlock(input_dim=dim, output_dim=(output_dim*self.num_of_mask_dim_to_add+self.num_of_mask_dim_to_add), kernel_size=1, stride=1, padding=0, norm='none', activation='tanh', pad_type=pad_type)]
        self.model = nn.Sequential(*self.model)

    def forward(self, x, im_in, return_mask=False):
        new_x = self.model(x)
        self.mask_s = ((torch.tanh(10 * new_x[:, (-1 * self.num_of_mask_dim_to_add):, :, :]) + 1) / 2)
        new_im = im_in
        curr_ind = 0
        for k in range(self.num_of_mask_dim_to_add):
            new_im_o = new_x[:, curr_ind:self.output_dim * (k + 1), :, :] ##
            curr_ind = self.output_dim * (k + 1)  ##
            mask = self.mask_s[:, k, :, :].unsqueeze(1).repeat(1, new_im.shape[1], 1, 1)
            new_im = (1 - mask) * new_im + mask * new_im_o

        if return_mask:
            if self.mask_s.shape[1] != 3:
                mask_s_prePixTot = torch.sum(self.mask_s, 1).unsqueeze(1).repeat(1, 3, 1, 1) / self.mask_s.shape[1]
                self.mask_s = mask_s_prePixTot
            return new_im, self.mask_s

        return new_im


##################################################################################
# Sequential Models
##################################################################################
class ResBlocks(nn.Module):
    def __init__(self, num_blocks, dim, norm='in', activation='relu', pad_type='zero'):
        super(ResBlocks, self).__init__()
        self.model = []
        for i in range(num_blocks):
            self.model += [ResBlock(dim, norm=norm, activation=activation, pad_type=pad_type)]
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x)

class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, dim, n_blk, norm='none', activ='relu'):
        super(MLP, self).__init__()
        self.model = []
        self.model += [LinearBlock(input_dim, dim, norm=norm, activation=activ)]
        for i in range(n_blk - 2):
            self.model += [LinearBlock(dim, dim, norm=norm, activation=activ)]
        self.model += [LinearBlock(dim, output_dim, norm='none', activation='none')]  # no output activations
        self.model = nn.Sequential(*self.model)

    def forward(self, x):
        return self.model(x.view(x.size(0), -1))

##################################################################################
# Basic Blocks
##################################################################################
class ResBlock(nn.Module):
    def __init__(self, dim, norm='in', activation='relu', pad_type='zero'):
        super(ResBlock, self).__init__()

        model = []
        model += [Conv2dBlock(dim ,dim, 3, 1, 1, norm=norm, activation=activation, pad_type=pad_type)]
        model += [Conv2dBlock(dim ,dim, 3, 1, 1, norm=norm, activation='none', pad_type=pad_type)]
        self.model = nn.Sequential(*model)

    def forward(self, x):
        residual = x
        out = self.model(x)
        out += residual
        return out

class Conv2dBlock(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, stride,
                 padding=0, norm='none', activation='relu', pad_type='zero'):
        super(Conv2dBlock, self).__init__()
        self.use_bias = True
        # initialize padding
        if pad_type == 'reflect':
            self.pad = nn.ReflectionPad2d(padding)
        elif pad_type == 'replicate':
            self.pad = nn.ReplicationPad2d(padding)
        elif pad_type == 'zero':
            self.pad = nn.ZeroPad2d(padding)
        else:
            assert 0, "Unsupported padding type: {}".format(pad_type)

        # initialize normalization
        norm_dim = output_dim
        if norm == 'bn':
            self.norm = nn.BatchNorm2d(norm_dim)
        elif norm == 'in':
            self.norm = nn.InstanceNorm2d(norm_dim)
        elif norm == 'ln':
            self.norm = LayerNorm(norm_dim)
        elif norm == 'adain':
            self.norm = AdaptiveInstanceNorm2d(norm_dim)
        elif norm == 'none' or norm == 'sn':
            self.norm = None
        else:
            assert 0, "Unsupported normalization: {}".format(norm)

        # initialize activation
        if activation == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU(0.2, inplace=True)
        elif activation == 'prelu':
            self.activation = nn.PReLU()
        elif activation == 'selu':
            self.activation = nn.SELU(inplace=True)
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'none':
            self.activation = None
        else:
            assert 0, "Unsupported activation: {}".format(activation)

        # initialize convolution
        if norm == 'sn':
            self.conv = SpectralNorm(nn.Conv2d(input_dim, output_dim, kernel_size, stride, bias=self.use_bias))
        else:
            self.conv = nn.Conv2d(input_dim, output_dim, kernel_size, stride, bias=self.use_bias)

    def forward(self, x):
        x = self.conv(self.pad(x))
        if self.norm:
            x = self.norm(x)
        if self.activation:
            x = self.activation(x)
        return x

class LinearBlock(nn.Module):
    def __init__(self, input_dim, output_dim, norm='none', activation='relu'):
        super(LinearBlock, self).__init__()
        use_bias = True
        # initialize fully connected layer
        if norm == 'sn':
            self.fc = SpectralNorm(nn.Linear(input_dim, output_dim, bias=use_bias))
        else:
            self.fc = nn.Linear(input_dim, output_dim, bias=use_bias)

        # initialize normalization
        norm_dim = output_dim
        if norm == 'bn':
            self.norm = nn.BatchNorm1d(norm_dim)
        elif norm == 'in':
            self.norm = nn.InstanceNorm1d(norm_dim)
        elif norm == 'ln':
            self.norm = LayerNorm(norm_dim)
        elif norm == 'none' or norm == 'sn':
            self.norm = None
        else:
            assert 0, "Unsupported normalization: {}".format(norm)

        # initialize activation
        if activation == 'relu':
            self.activation = nn.ReLU(inplace=True)
        elif activation == 'lrelu':
            self.activation = nn.LeakyReLU(0.2, inplace=True)
        elif activation == 'prelu':
            self.activation = nn.PReLU()
        elif activation == 'selu':
            self.activation = nn.SELU(inplace=True)
        elif activation == 'tanh':
            self.activation = nn.Tanh()
        elif activation == 'none':
            self.activation = None
        else:
            assert 0, "Unsupported activation: {}".format(activation)

    def forward(self, x):
        out = self.fc(x)
        if self.norm:
            out = self.norm(out)
        if self.activation:
            out = self.activation(out)
        return out

##################################################################################
# VGG network definition
##################################################################################
class Vgg16(nn.Module):
    def __init__(self):
        super(Vgg16, self).__init__()
        self.conv1_1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)

        self.conv2_1 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1)

        self.conv3_1 = nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        self.conv4_1 = nn.Conv2d(256, 512, kernel_size=3, stride=1, padding=1)
        self.conv4_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv4_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

        self.conv5_1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv5_2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)
        self.conv5_3 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1)

    def forward(self, X):
        h = F.relu(self.conv1_1(X), inplace=True)
        h = F.relu(self.conv1_2(h), inplace=True)
        # relu1_2 = h
        h = F.max_pool2d(h, kernel_size=2, stride=2)

        h = F.relu(self.conv2_1(h), inplace=True)
        h = F.relu(self.conv2_2(h), inplace=True)
        # relu2_2 = h
        h = F.max_pool2d(h, kernel_size=2, stride=2)

        h = F.relu(self.conv3_1(h), inplace=True)
        h = F.relu(self.conv3_2(h), inplace=True)
        h = F.relu(self.conv3_3(h), inplace=True)
        # relu3_3 = h
        h = F.max_pool2d(h, kernel_size=2, stride=2)

        h = F.relu(self.conv4_1(h), inplace=True)
        h = F.relu(self.conv4_2(h), inplace=True)
        h = F.relu(self.conv4_3(h), inplace=True)
        # relu4_3 = h

        h = F.relu(self.conv5_1(h), inplace=True)
        h = F.relu(self.conv5_2(h), inplace=True)
        h = F.relu(self.conv5_3(h), inplace=True)
        relu5_3 = h

        return relu5_3
        # return [relu1_2, relu2_2, relu3_3, relu4_3]

##################################################################################
# Normalization layers
##################################################################################
class AdaptiveInstanceNorm2d(nn.Module):
    def __init__(self, num_features, eps=1e-5, momentum=0.1):
        super(AdaptiveInstanceNorm2d, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.momentum = momentum
        # weight and bias are dynamically assigned
        self.weight = None
        self.bias = None
        # just dummy buffers, not used
        self.register_buffer('running_mean', torch.zeros(num_features))
        self.register_buffer('running_var', torch.ones(num_features))

    def forward(self, x):
        assert self.weight is not None and self.bias is not None, "Please assign weight and bias before calling AdaIN!"
        b, c = x.size(0), x.size(1)
        running_mean = self.running_mean.repeat(b)
        running_var = self.running_var.repeat(b)

        # Apply instance norm
        x_reshaped = x.contiguous().view(1, b * c, *x.size()[2:])

        out = F.batch_norm(
            x_reshaped, running_mean, running_var, self.weight, self.bias,
            True, self.momentum, self.eps)

        return out.view(b, c, *x.size()[2:])

    def __repr__(self):
        return self.__class__.__name__ + '(' + str(self.num_features) + ')'


class LayerNorm(nn.Module):
    def __init__(self, num_features, eps=1e-5, affine=True):
        super(LayerNorm, self).__init__()
        self.num_features = num_features
        self.affine = affine
        self.eps = eps

        if self.affine:
            self.gamma = nn.Parameter(torch.Tensor(num_features).uniform_())
            self.beta = nn.Parameter(torch.zeros(num_features))

    def forward(self, x):
        shape = [-1] + [1] * (x.dim() - 1)
        # print(x.size())
        if x.size(0) == 1:
            # These two lines run much faster in pytorch 0.4 than the two lines listed below.
            mean = x.view(-1).mean().view(*shape)
            std = x.view(-1).std().view(*shape)
        else:
            mean = x.view(x.size(0), -1).mean(1).view(*shape)
            std = x.view(x.size(0), -1).std(1).view(*shape)

        x = (x - mean) / (std + self.eps)

        if self.affine:
            shape = [1, -1] + [1] * (x.dim() - 2)
            x = x * self.gamma.view(*shape) + self.beta.view(*shape)
        return x

def l2normalize(v, eps=1e-12):
    return v / (v.norm() + eps)


class SpectralNorm(nn.Module):
    """
    Based on the paper "Spectral Normalization for Generative Adversarial Networks" by Takeru Miyato, Toshiki Kataoka, Masanori Koyama, Yuichi Yoshida
    and the Pytorch implementation https://github.com/christiancosgrove/pytorch-spectral-normalization-gan
    """
    def __init__(self, module, name='weight', power_iterations=1):
        super(SpectralNorm, self).__init__()
        self.module = module
        self.name = name
        self.power_iterations = power_iterations
        if not self._made_params():
            self._make_params()

    def _update_u_v(self):
        u = getattr(self.module, self.name + "_u")
        v = getattr(self.module, self.name + "_v")
        w = getattr(self.module, self.name + "_bar")

        height = w.data.shape[0]
        for _ in range(self.power_iterations):
            v.data = l2normalize(torch.mv(torch.t(w.view(height,-1).data), u.data))
            u.data = l2normalize(torch.mv(w.view(height,-1).data, v.data))
        sigma = u.dot(w.view(height, -1).mv(v))
        setattr(self.module, self.name, w / sigma.expand_as(w))

    def _made_params(self):
        try:
            u = getattr(self.module, self.name + "_u")
            v = getattr(self.module, self.name + "_v")
            w = getattr(self.module, self.name + "_bar")
            return True
        except AttributeError:
            return False


    def _make_params(self):
        w = getattr(self.module, self.name)

        height = w.data.shape[0]
        width = w.view(height, -1).data.shape[1]

        u = nn.Parameter(w.data.new(height).normal_(0, 1), requires_grad=False)
        v = nn.Parameter(w.data.new(width).normal_(0, 1), requires_grad=False)
        u.data = l2normalize(u.data)
        v.data = l2normalize(v.data)
        w_bar = nn.Parameter(w.data)

        del self.module._parameters[self.name]

        self.module.register_parameter(self.name + "_u", u)
        self.module.register_parameter(self.name + "_v", v)
        self.module.register_parameter(self.name + "_bar", w_bar)


    def forward(self, *args):
        self._update_u_v()
        return self.module.forward(*args)


# utils.py


In [4]:
"""
Copyright (C) 2018 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""
# from torch.utils.serialization import load_lua
import torchfile
from torch.utils.data import DataLoader
from torchvision.models import inception_v3
# from networks import Vgg16
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import transforms
# from data import ImageFilelist, ImageFolder, ImageFolder_with_subfolders
import torch
import torch.nn as nn
import os
import math
import torchvision.utils as vutils
import yaml
import numpy as np
import torch.nn.init as init
import time
# Methods
# get_all_data_loaders      : primary data loader interface (load trainA, testA, trainB, testB)
# get_data_loader_list      : list-based data loader
# get_data_loader_folder    : folder-based data loader
# get_config                : load yaml file
# eformat                   :
# write_2images             : save output image
# prepare_sub_folder        : create checkpoints and images folders for saving outputs
# write_one_row_html        : write one row of the html file for output images
# write_html                : create the html file.
# write_loss
# slerp
# get_slerp_interp
# get_model_list
# load_vgg16
# load_inception
# vgg_preprocess
# get_scheduler
# weights_init

def get_all_data_loaders(conf):
    batch_size = conf['batch_size']
    num_workers = conf['num_workers']
    if 'new_size' in conf:
        new_size_a = new_size_b = conf['new_size']
    else:
        new_size_a = conf['new_size_a']
        new_size_b = conf['new_size_b']
    height = conf['crop_image_height']
    width = conf['crop_image_width']

    train_loader_a, train_loader_b, test_loader_a, test_loader_b = [], [], [], []
    if 'data_root' in conf:
        if 'inbalenceDataSets' in conf:
            if (conf['inbalenceDataSets']['imbalance_sub_dataset']):

                train_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'],  'trainA'), batch_size, True,
                                                             new_size_a, height, width, num_workers, True, config=conf,
                                                             is_data_A=True, ratio_1_to_2=conf['inbalenceDataSets']['ratio_A_1_to_2']))
                test_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testA'), batch_size, False,
                                                            new_size_a, new_size_a, new_size_a, num_workers, True,
                                                            config=conf, is_data_A=True, ratio_1_to_2=conf['inbalenceDataSets']['ratio_A_1_to_2']))
                train_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'trainB'), batch_size, True,
                                                             new_size_b, height, width, num_workers, True, config=conf,
                                                             is_data_A=False, ratio_1_to_2=conf['inbalenceDataSets']['ratio_B_1_to_2']))
                test_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testB'), batch_size, False,
                                                            new_size_b, new_size_b, new_size_b, num_workers, True,
                                                            config=conf, is_data_A=False, ratio_1_to_2=conf['inbalenceDataSets']['ratio_B_1_to_2']))
            else:
                train_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'], 'trainA'), batch_size, True,
                                                        new_size_a, height, width, num_workers, True, config=conf,
                                                        is_data_A=True))
                test_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testA'), batch_size, False,
                                                       new_size_a, new_size_a, new_size_a, num_workers, True,
                                                       config=conf, is_data_A=True))
                train_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'trainB'), batch_size, True,
                                                        new_size_b, height, width, num_workers, True, config=conf,
                                                        is_data_A=False))
                test_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testB'), batch_size, False,
                                                       new_size_b, new_size_b, new_size_b, num_workers, True,
                                                       config=conf, is_data_A=False))
        else:
            train_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'], 'trainA'), batch_size, True,
                                                  new_size_a, height, width, num_workers, True, config=conf, is_data_A=True))
            test_loader_a.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testA'), batch_size, False,
                                                 new_size_a, new_size_a, new_size_a, num_workers, True, config=conf, is_data_A=True))
            train_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'trainB'), batch_size, True,
                                                  new_size_b, height, width, num_workers, True, config=conf, is_data_A=False))
            test_loader_b.append(get_data_loader_folder(os.path.join(conf['data_root'], 'testB'), batch_size, False,
                                                 new_size_b, new_size_b, new_size_b, num_workers, True, config=conf, is_data_A=False))
    else:
        train_loader_a.append(get_data_loader_list(conf['data_folder_train_a'], conf['data_list_train_a'], batch_size, True,
                                                new_size_a, height, width, num_workers, True, is_data_A=True))
        test_loader_a.append(get_data_loader_list(conf['data_folder_test_a'], conf['data_list_test_a'], batch_size, False,
                                                new_size_a, new_size_a, new_size_a, num_workers, True, is_data_A=True))
        train_loader_b.append(get_data_loader_list(conf['data_folder_train_b'], conf['data_list_train_b'], batch_size, True,
                                                new_size_b, height, width, num_workers, True, is_data_A=False))
        test_loader_b.append(get_data_loader_list(conf['data_folder_test_b'], conf['data_list_test_b'], batch_size, False,
                                                new_size_b, new_size_b, new_size_b, num_workers, True, is_data_A=False))
    return train_loader_a, train_loader_b, test_loader_a, test_loader_b

def get_data_loader_list(root, file_list, batch_size, train, new_size=None,
                           height=256, width=256, num_workers=4, crop=True):
    transform_list = [transforms.ToTensor(),
                      transforms.Normalize((0.5, 0.5, 0.5),
                                           (0.5, 0.5, 0.5))]
    transform_list = [transforms.RandomCrop((height, width))] + transform_list if crop else transform_list
    transform_list = [transforms.Resize(new_size)] + transform_list if new_size is not None else transform_list
    transform_list = [transforms.RandomHorizontalFlip()] + transform_list if train else transform_list
    transform = transforms.Compose(transform_list)
    dataset = ImageFilelist(root, file_list, transform=transform)
    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=train, drop_last=True, num_workers=num_workers)
    return loader

def dim3to1(x):
    x = torch.sum(x, 0).unsqueeze(0)
    return x

def get_data_loader_folder(input_folder, batch_size, train, new_size=None,
                           height=256, width=256, num_workers=4, crop=True, config=None, is_data_A=None, ratio_1_to_2=None):
    transform_list = [transforms.ToTensor(),
                      transforms.Normalize((0.5, 0.5, 0.5),
                                           (0.5, 0.5, 0.5))]
    transform_list = [transforms.CenterCrop((new_size))] + transform_list if not train else transform_list
    transform_list = [transforms.RandomCrop((height, width))] + transform_list if crop else transform_list
    transform_list = [transforms.Resize(new_size)] + transform_list if new_size is not None else transform_list
    transform_list = (transform_list + [dim3to1]) if config['input_dim_a'] == 1 and is_data_A else transform_list
    transform_list = (transform_list + [dim3to1]) if config['input_dim_b'] == 1 and not is_data_A else transform_list
    if config is not None:
        if config['do_HorizontalFlip']:
            transform_list = [transforms.RandomHorizontalFlip()] + transform_list if train else transform_list
    if config is not None:
        if config['do_RandomResizedCrop']:
            ratio_max = eval(config['RandomResizedCrop_ratio_max'])
            ratio_min = eval(config['RandomResizedCrop_ratio_min'])
            transform_list = [transforms.RandomResizedCrop(size=new_size, scale=(config['RandomResizedCrop_scale_min'], config['RandomResizedCrop_scale_max']),
                                                           ratio=(ratio_min, ratio_max))] + transform_list if train else transform_list
    if config is not None:
        if config['do_VerticalFlip']:
            transform_list = [transforms.RandomVerticalFlip(p=0.35)] + transform_list if train else transform_list

    if config is not None:
        if is_data_A is not None:
            if config['do_ColorJitter_A'] and is_data_A:
                transform_list = [transforms.ColorJitter(brightness=config['ColorJitter_brightness'], contrast=config['ColorJitter_contrast'], saturation=config['ColorJitter_saturation'], hue=config['ColorJitter_hue'])] + transform_list if train else transform_list
            elif config['do_ColorJitter_B'] and not is_data_A:
                transform_list = [transforms.ColorJitter(brightness=config['ColorJitter_brightness'], contrast=config['ColorJitter_contrast'], saturation=config['ColorJitter_saturation'], hue=config['ColorJitter_hue'])] + transform_list if train else transform_list
        else:
            if config['do_ColorJitter']:
                transform_list = [transforms.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=config['ColorJitter_hue'])] + transform_list if train else transform_list

    if config is not None:
        if config['do_RandomGrayscale']:
            transform_list = [transforms.RandomGrayscale(p=config['RandomGrayscale_P'])] + transform_list if train else transform_list

    if config is not None:
        if config['do_RandomRotation']:
            transform_list = [transforms.RandomRotation(degrees=config['RandomRotation_degree'], expand=False)] + transform_list if train else transform_list

    if config is not None:
        if config['do_RandomAffine']:
            config['RandomAffine_translate_w']
            transform_list = [transforms.RandomAffine(translate=(config['RandomAffine_translate_h'], config['RandomAffine_translate_w']), degrees=0, scale=(0.9, 1.1))] + transform_list if train else transform_list

    if config is not None:
        if config['do_RandomPerspective']:
            transform_list = [transforms.RandomPerspective(distortion_scale=0.35, p=0.5)] + transform_list if train else transform_list

    transform = transforms.Compose(transform_list)
    if ratio_1_to_2 is None:
        dataset = ImageFolder(input_folder, transform=transform)
    else:
        input_folder_1 = os.path.join(input_folder, '1')
        input_folder_2 = os.path.join(input_folder, '2')
        dataset = ImageFolder_with_subfolders(root1=input_folder_1, root2=input_folder_2, ratio_1_to_2=ratio_1_to_2, transform=transform)

    loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=train, drop_last=True, num_workers=num_workers)
    return loader

def get_config(config):
    with open(config, 'r') as stream:
        return yaml.safe_load(stream)

def eformat(f, prec):
    s = "%.*e"%(prec, f)
    mantissa, exp = s.split('e')
    # add 1 to digits as 1 is taken by sign +/-
    return "%se%d"%(mantissa, int(exp))

def __write_images(image_outputs, display_image_num, file_name):
    if image_outputs[0].shape[1]>3:
        rand_disp_index_s = range(image_outputs[0].shape[1])
        rand_disp_index_s = np.random.permutation(rand_disp_index_s)
        rand_disp_index_s = rand_disp_index_s[:3]
        rand_disp_index_s = np.sort(rand_disp_index_s)
        image_outputs = [images[:, rand_disp_index_s, :, :] if images.shape[1] > 3 else images for images in image_outputs] # TODO tmp
    else:
        image_outputs = [images.expand(-1, 3, -1, -1) for images in image_outputs] # expand gray-scale images to 3 channels
    image_tensor = torch.cat([images[:display_image_num] for images in image_outputs], 0)
    image_grid = vutils.make_grid(image_tensor.data, nrow=display_image_num, padding=0, normalize=True)

    vutils.save_image(image_grid, file_name, nrow=1)

    # from PIL import Image
    grid = vutils.make_grid(image_grid, nrow=8, padding=2, pad_value=0,
                     normalize=False, range=None, scale_each=False)


    return grid.to('cpu', torch.uint8).numpy()

def write_2images(image_outputs, display_image_num, image_directory, postfix, do_a2b=True, do_b2a=True):
    n = len(image_outputs)
    gen_a2b_im, gen_b2a_im = [], []
    if do_a2b:
        gen_a2b_im = __write_images(image_outputs[0:n//2], display_image_num, '%s/gen_a2b_%s.jpg' % (image_directory, postfix)) if do_a2b else None
    if do_b2a:
        gen_b2a_im = __write_images(image_outputs[n//2:n], display_image_num, '%s/gen_b2a_%s.jpg' % (image_directory, postfix)) if do_b2a else None
    return gen_a2b_im, gen_b2a_im

def prepare_sub_folder(output_directory):
    image_directory = os.path.join(output_directory, 'images')
    if not os.path.exists(image_directory):
        print("Creating directory: {}".format(image_directory))
        os.makedirs(image_directory)
    checkpoint_directory = os.path.join(output_directory, 'checkpoints')
    if not os.path.exists(checkpoint_directory):
        print("Creating directory: {}".format(checkpoint_directory))
        os.makedirs(checkpoint_directory)
    checkpoint_log = os.path.join(output_directory, 'log')
    if not os.path.exists(checkpoint_log):
        print("Creating directory: {}".format(checkpoint_log))
        os.makedirs(checkpoint_log)
    return checkpoint_directory, image_directory, checkpoint_log

def write_one_row_html(html_file, iterations, img_filename, all_size):
    html_file.write("<h3>iteration [%d] (%s)</h3>" % (iterations,img_filename.split('/')[-1]))
    html_file.write("""
        <p><a href="%s">
          <img src="%s" style="width:%dpx">
        </a><br>
        <p>
        """ % (img_filename, img_filename, all_size))
    return

def write_html(filename, iterations, image_save_iterations, image_directory, all_size=1536, do_a2b=True, do_b2a=True):
    html_file = open(filename, "w")
    html_file.write('''
    <!DOCTYPE html>
    <html>
    <head>
      <title>Experiment name = %s</title>
      <meta http-equiv="refresh" content="30">
    </head>
    <body>
    ''' % os.path.basename(filename))
    html_file.write("<h3>current</h3>")
    if do_a2b:
        write_one_row_html(html_file, iterations, '%s/gen_a2b_train_current.jpg' % (image_directory), all_size)
    if do_b2a:
        write_one_row_html(html_file, iterations, '%s/gen_b2a_train_current.jpg' % (image_directory), all_size)
    for j in range(iterations, image_save_iterations-1, -1):
        if j % image_save_iterations == 0:
            if do_a2b:
                write_one_row_html(html_file, j, '%s/gen_a2b_test_%08d.jpg' % (image_directory, j), all_size)
            if do_b2a:
                write_one_row_html(html_file, j, '%s/gen_b2a_test_%08d.jpg' % (image_directory, j), all_size)
            if do_a2b:
                write_one_row_html(html_file, j, '%s/gen_a2b_train_%08d.jpg' % (image_directory, j), all_size)
            if do_b2a:
                write_one_row_html(html_file, j, '%s/gen_b2a_train_%08d.jpg' % (image_directory, j), all_size)
    html_file.write("</body></html>")
    html_file.close()

def write_loss(iterations, trainer, train_writer):
    members = [attr for attr in dir(trainer)
               if not callable(getattr(trainer, attr)) and not attr.startswith("__") and ('loss' in attr or 'grad' in attr or 'conf' in attr or 'nwd' in attr or 'do' in attr)]
    for m in members:
        tmpatter = getattr(trainer, m)
        if m.endswith('_conf'):
            m = 'configs/' + m
        if '_a_' in m or '_b_' in m or '_ab_' in m or '_ba_' in m:
            ind = m.find('_a_')
            ind = ind if ind != -1 else m.find('_b_')
            ind = ind if ind != -1 else m.find('_ab_')
            ind = ind if ind != -1 else m.find('_ba_')

            m = m[0:ind] + '/' + m
        if type(tmpatter) is bool:
            tmpatter = 1 if tmpatter else 0
            m = 'configs/' + m

        if type(tmpatter) is list:
            tmpList = tmpatter
            tmpScal = {}
            for i, listItem in enumerate(tmpList):
                if type(listItem) is torch.Tensor:
                    tmpScal.update({str(i): listItem.data.cpu().numpy()})
                else:
                    tmpScal.update({m + '_' + str(i): listItem})
            train_writer.add_scalars(m, tmpScal, iterations + 1)
        else:
            train_writer.add_scalar(m, tmpatter, iterations + 1)


def slerp(val, low, high):
    """
    original: Animating Rotation with Quaternion Curves, Ken Shoemake
    https://arxiv.org/abs/1609.04468
    Code: https://github.com/soumith/dcgan.torch/issues/14, Tom White
    """
    omega = np.arccos(np.dot(low / np.linalg.norm(low), high / np.linalg.norm(high)))
    so = np.sin(omega)
    return np.sin((1.0 - val) * omega) / so * low + np.sin(val * omega) / so * high

def get_slerp_interp(nb_latents, nb_interp, z_dim):
    """
    modified from: PyTorch inference for "Progressive Growing of GANs" with CelebA snapshot
    https://github.com/ptrblck/prog_gans_pytorch_inference
    """

    latent_interps = np.empty(shape=(0, z_dim), dtype=np.float32)
    for _ in range(nb_latents):
        low = np.random.randn(z_dim)
        high = np.random.randn(z_dim)  # low + np.random.randn(512) * 0.7
        interp_vals = np.linspace(0, 1, num=nb_interp)
        latent_interp = np.array([slerp(v, low, high) for v in interp_vals],
                                 dtype=np.float32)
        latent_interps = np.vstack((latent_interps, latent_interp))

    return latent_interps[:, :, np.newaxis, np.newaxis]

# Get model list for resume
def get_model_list(dirname, key):
    if os.path.exists(dirname) is False:
        return None
    gen_models = [os.path.join(dirname, f) for f in os.listdir(dirname) if
                  os.path.isfile(os.path.join(dirname, f)) and key in f and ".pt" in f]
    if gen_models is None:
        return None
    gen_models.sort()
    if len(gen_models)>0:
        last_model_name = gen_models[-1]
    else:
        last_model_name = None
    return last_model_name

def load_vgg16(model_dir):
    """ Use the model from https://github.com/abhiskk/fast-neural-style/blob/master/neural_style/utils.py """
    if not os.path.exists(model_dir):
        os.mkdir(model_dir)
    if not os.path.exists(os.path.join(model_dir, 'vgg16.weight')):
        if not os.path.exists(os.path.join(model_dir, 'vgg16.t7')):
            os.system('wget https://www.dropbox.com/s/76l3rt4kyi3s8x7/vgg16.t7?dl=1 -O ' + os.path.join(model_dir, 'vgg16.t7'))
        # vgglua = load_lua(os.path.join(model_dir, 'vgg16.t7'))
        vgglua = torchfile.load(os.path.join(model_dir, 'vgg16.t7'))

        vgg = Vgg16()
        for (src, dst) in zip(vgglua.parameters()[0], vgg.parameters()):
            dst.data[:] = src
        torch.save(vgg.state_dict(), os.path.join(model_dir, 'vgg16.weight'))
    vgg = Vgg16()
    vgg.load_state_dict(torch.load(os.path.join(model_dir, 'vgg16.weight')))
    return vgg

def load_inception(model_path, pretrained=False):
    if model_path is not None:
        state_dict = torch.load(model_path)
    model = inception_v3(pretrained=pretrained, transform_input=True)
    model.aux_logits = False
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, state_dict['fc.weight'].size(0))
    model.load_state_dict(state_dict)
    for param in model.parameters():
        param.requires_grad = False
    return model

def vgg_preprocess(batch):
    tensortype = type(batch.data)
    (r, g, b) = torch.chunk(batch, 3, dim = 1)
    batch = torch.cat((b, g, r), dim = 1) # convert RGB to BGR
    batch = (batch + 1) * 255 * 0.5 # [-1, 1] -> [0, 255]
    mean = tensortype(batch.data.size()).cuda()
    mean[:, 0, :, :] = 103.939
    mean[:, 1, :, :] = 116.779
    mean[:, 2, :, :] = 123.680
    batch = batch.sub(Variable(mean)) # subtract mean
    return batch

def get_scheduler(optimizer, hyperparameters, iterations=-1):
    if 'lr_policy' not in hyperparameters or hyperparameters['lr_policy'] == 'constant':
        scheduler = None # constant scheduler
    elif hyperparameters['lr_policy'] == 'step':
        scheduler = lr_scheduler.StepLR(optimizer, step_size=hyperparameters['step_size'],
                                        gamma=hyperparameters['gamma'], last_epoch=iterations)
    else:
        return NotImplementedError('learning rate policy [%s] is not implemented', hyperparameters['lr_policy'])
    return scheduler

def weights_init(init_type='gaussian'):
    def init_fun(m):
        classname = m.__class__.__name__
        if (classname.find('Conv') == 0 or classname.find('Linear') == 0) and hasattr(m, 'weight'):
            # print m.__class__.__name__
            if init_type == 'gaussian':
                init.normal_(m.weight.data, 0.0, 0.02)
            elif init_type == 'xavier':
                init.xavier_normal_(m.weight.data, gain=math.sqrt(2))
            elif init_type == 'kaiming':
                init.kaiming_normal_(m.weight.data, a=0, mode='fan_in')
            elif init_type == 'orthogonal':
                init.orthogonal_(m.weight.data, gain=math.sqrt(2))
            elif init_type == 'default':
                pass
            else:
                assert 0, "Unsupported initialization: {}".format(init_type)
            if hasattr(m, 'bias') and m.bias is not None:
                init.constant_(m.bias.data, 0.0)

    return init_fun

class Timer:
    def __init__(self, msg):
        self.msg = msg
        self.start_time = None

    def __enter__(self):
        self.start_time = time.time()

    def __exit__(self, exc_type, exc_value, exc_tb):
        print(self.msg % (time.time() - self.start_time))

def pytorch03_to_pytorch04(state_dict_base, trainer_name):
    def __conversion_core(state_dict_base, trainer_name):
        state_dict = state_dict_base.copy()
        if trainer_name == 'MUNIT':
            for key, value in state_dict_base.items():
                if key.endswith(('enc_content.model.0.norm.running_mean',
                                 'enc_content.model.0.norm.running_var',
                                 'enc_content.model.1.norm.running_mean',
                                 'enc_content.model.1.norm.running_var',
                                 'enc_content.model.2.norm.running_mean',
                                 'enc_content.model.2.norm.running_var',
                                 'enc_content.model.3.model.0.model.1.norm.running_mean',
                                 'enc_content.model.3.model.0.model.1.norm.running_var',
                                 'enc_content.model.3.model.0.model.0.norm.running_mean',
                                 'enc_content.model.3.model.0.model.0.norm.running_var',
                                 'enc_content.model.3.model.1.model.1.norm.running_mean',
                                 'enc_content.model.3.model.1.model.1.norm.running_var',
                                 'enc_content.model.3.model.1.model.0.norm.running_mean',
                                 'enc_content.model.3.model.1.model.0.norm.running_var',
                                 'enc_content.model.3.model.2.model.1.norm.running_mean',
                                 'enc_content.model.3.model.2.model.1.norm.running_var',
                                 'enc_content.model.3.model.2.model.0.norm.running_mean',
                                 'enc_content.model.3.model.2.model.0.norm.running_var',
                                 'enc_content.model.3.model.3.model.1.norm.running_mean',
                                 'enc_content.model.3.model.3.model.1.norm.running_var',
                                 'enc_content.model.3.model.3.model.0.norm.running_mean',
                                 'enc_content.model.3.model.3.model.0.norm.running_var',
                                 )):
                    del state_dict[key]
        else:
            def __conversion_core(state_dict_base):
                state_dict = state_dict_base.copy()
                for key, value in state_dict_base.items():
                    if key.endswith(('enc.model.0.norm.running_mean',
                                     'enc.model.0.norm.running_var',
                                     'enc.model.1.norm.running_mean',
                                     'enc.model.1.norm.running_var',
                                     'enc.model.2.norm.running_mean',
                                     'enc.model.2.norm.running_var',
                                     'enc.model.3.model.0.model.1.norm.running_mean',
                                     'enc.model.3.model.0.model.1.norm.running_var',
                                     'enc.model.3.model.0.model.0.norm.running_mean',
                                     'enc.model.3.model.0.model.0.norm.running_var',
                                     'enc.model.3.model.1.model.1.norm.running_mean',
                                     'enc.model.3.model.1.model.1.norm.running_var',
                                     'enc.model.3.model.1.model.0.norm.running_mean',
                                     'enc.model.3.model.1.model.0.norm.running_var',
                                     'enc.model.3.model.2.model.1.norm.running_mean',
                                     'enc.model.3.model.2.model.1.norm.running_var',
                                     'enc.model.3.model.2.model.0.norm.running_mean',
                                     'enc.model.3.model.2.model.0.norm.running_var',
                                     'enc.model.3.model.3.model.1.norm.running_mean',
                                     'enc.model.3.model.3.model.1.norm.running_var',
                                     'enc.model.3.model.3.model.0.norm.running_mean',
                                     'enc.model.3.model.3.model.0.norm.running_var',

                                     'dec.model.0.model.0.model.1.norm.running_mean',
                                     'dec.model.0.model.0.model.1.norm.running_var',
                                     'dec.model.0.model.0.model.0.norm.running_mean',
                                     'dec.model.0.model.0.model.0.norm.running_var',
                                     'dec.model.0.model.1.model.1.norm.running_mean',
                                     'dec.model.0.model.1.model.1.norm.running_var',
                                     'dec.model.0.model.1.model.0.norm.running_mean',
                                     'dec.model.0.model.1.model.0.norm.running_var',
                                     'dec.model.0.model.2.model.1.norm.running_mean',
                                     'dec.model.0.model.2.model.1.norm.running_var',
                                     'dec.model.0.model.2.model.0.norm.running_mean',
                                     'dec.model.0.model.2.model.0.norm.running_var',
                                     'dec.model.0.model.3.model.1.norm.running_mean',
                                     'dec.model.0.model.3.model.1.norm.running_var',
                                     'dec.model.0.model.3.model.0.norm.running_mean',
                                     'dec.model.0.model.3.model.0.norm.running_var',
                                     )):
                        del state_dict[key]
        return state_dict

    state_dict = dict()
    state_dict['a'] = __conversion_core(state_dict_base['a'], trainer_name)
    state_dict['b'] = __conversion_core(state_dict_base['b'], trainer_name)
    return state_dict



In [5]:
"""
Copyright (C) 2017 NVIDIA Corporation.  All rights reserved.
Licensed under the CC BY-NC-SA 4.0 license (https://creativecommons.org/licenses/by-nc-sa/4.0/legalcode).
"""
# from networks import AdaINGen, MsImageDis, MsImageDisCouncil
# from utils import weights_init, get_model_list, vgg_preprocess, load_vgg16, get_scheduler
from torch.autograd import Variable
import torch
import torch.nn as nn
import os
import random
import threading
from multiprocessing.pool import ThreadPool
import warnings
from collections import deque
import numpy as np
import torchvision.transforms.functional as TF
from scipy import ndimage

class Council_Trainer(nn.Module):
    def __init__(self, hyperparameters, cuda_device='cuda:0'):
        super(Council_Trainer, self).__init__()
        lr = hyperparameters['lr']
        # Initiate the networks
        self.council_size = hyperparameters['council']['council_size']
        self.council_size_conf = self.council_size
        self.gen_a2b_s = []
        self.gen_b2a_s = []
        self.dis_a2b_s = []
        self.dis_b2a_s = []
        self.do_dis_council = hyperparameters['council_w'] != 0
        self.do_ads_council_loss = hyperparameters['council_abs_w'] != 0
        self.numberOfCouncil_dis_relative_iteration_conf = hyperparameters['council']['numberOfCouncil_dis_relative_iteration']
        self.discriminetro_less_style_by_conf = hyperparameters['council']['discriminetro_less_style_by']
        self.cuda_device = cuda_device

        # all varible with '_conf' at the end will be saved and displayed in tensorboard logs
        self.recon_x_w_conf = hyperparameters['recon_x_w']
        self.recon_c_w_conf = hyperparameters['recon_c_w']
        self.recon_s_w_conf = hyperparameters['recon_s_w']
        self.recon_x_cyc_w_conf = hyperparameters['recon_x_cyc_w']
        self.gan_w_conf = hyperparameters['gan_w']
        self.vgg_w_conf = hyperparameters['vgg_w']
        self.abs_beginning_end_w_conf = hyperparameters['abs_beginning_end']
        self.flipOnOff_On_iteration_conf = hyperparameters['council']['flipOnOff_On_iteration']
        self.flipOnOff_Off_iteration_conf = hyperparameters['council']['flipOnOff_Off_iteration']
        self.flipOnOff_Off_iteration_conf = hyperparameters['council']['flipOnOff_start_with']
        self.council_abs_w_conf = hyperparameters['council_abs_w']
        self.council_w_conf = hyperparameters['council_w']
        self.council_start_at_iter_conf = hyperparameters['council']['council_start_at_iter']
        self.focus_loss_start_at_iter_conf = hyperparameters['focus_loss']['focus_loss_start_at_iter']
        self.mask_zero_or_one_w_conf = hyperparameters['mask_zero_or_one_w']
        self.mask_zero_or_one_center_conf = hyperparameters['focus_loss']['mask_zero_or_one_center']
        self.mask_zero_or_one_epsilon_conf = hyperparameters['focus_loss']['mask_zero_or_one_epsilon']
        self.mask_total_w_conf = hyperparameters['mask_total_w']
        self.mask_tv_w_conf = hyperparameters['mask_tv_w']
        self.batch_size_conf = hyperparameters['batch_size']
        self.do_w_loss_matching = hyperparameters['do_w_loss_matching']
        self.do_w_loss_matching_focus = hyperparameters['focus_loss']['do_w_loss_matching_focus']
        self.los_matching_hist_size_conf = hyperparameters['loss_matching_hist_size']
        self.do_a2b_conf = hyperparameters['do_a2b']
        self.do_b2a_conf = hyperparameters['do_b2a']
        self.w_match_b2a_conf = 1
        self.w_match_a2b_conf = 1
        self.w_match_focus_a2b_conf = 1
        self.w_match_focus_b2a_conf = 1
        self.w_match_focus_zero_one_a2b_conf = 1
        self.w_match_focus_zero_one_b2a_conf = 1

        if self.do_a2b_conf:
            self.los_hist_gan_a2b_s = []
            self.los_hist_council_a2b_s = []
            self.los_hist_focus_a2b_s = []
            self.los_hist_focus_zero_one_a2b_s = []
        if self.do_b2a_conf:
            self.los_hist_gan_b2a_s = []
            self.los_hist_council_b2a_s = []
            self.los_hist_focus_b2a_s = []
            self.los_hist_focus_zero_one_b2a_s = []

        for ind in range(self.council_size):
            if self.do_a2b_conf:
                self.los_hist_gan_a2b_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_council_a2b_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_focus_a2b_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_focus_zero_one_a2b_s.append(deque(np.ones(self.los_matching_hist_size_conf)))

            if self.do_b2a_conf:
                self.los_hist_gan_b2a_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_council_b2a_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_focus_b2a_s.append(deque(np.ones(self.los_matching_hist_size_conf)))
                self.los_hist_focus_zero_one_b2a_s.append(deque(np.ones(self.los_matching_hist_size_conf)))

        self.do_council_loss = None

        if self.do_dis_council:
            self.dis_council_a2b_s = []
            self.dis_council_b2a_s = []

        # defining all the networks
        for i in range(self.council_size):
            if self.do_a2b_conf:
                self.gen_a2b_s.append(
                    AdaINGen(hyperparameters['input_dim_a'], hyperparameters['gen'], cuda_device=self.cuda_device))  # auto-encoder for domain a2b
                self.dis_a2b_s.append(
                    MsImageDis(hyperparameters['input_dim_a'], hyperparameters['dis'], cuda_device=self.cuda_device))  # discriminator for domain a2b
                if self.do_dis_council:
                    self.dis_council_a2b_s.append(
                        MsImageDisCouncil(hyperparameters['input_dim_a'],
                                          hyperparameters['dis'], cuda_device=self.cuda_device))  # council discriminator for domain a2b
            if self.do_b2a_conf:
                self.gen_b2a_s.append(
                    AdaINGen(hyperparameters['input_dim_b'], hyperparameters['gen'], cuda_device=self.cuda_device))  # auto-encoder for domain b
                self.dis_b2a_s.append(
                    MsImageDis(hyperparameters['input_dim_b'], hyperparameters['dis'], cuda_device=self.cuda_device))  # discriminator for domain b
                if self.do_dis_council:
                    self.dis_council_b2a_s.append(#
                        MsImageDisCouncil(hyperparameters['input_dim_b'],
                                          hyperparameters['dis'], cuda_device=self.cuda_device))  # discriminator for domain b

        self.instancenorm = nn.InstanceNorm2d(512, affine=False)
        self.style_dim = hyperparameters['gen']['style_dim']

        if self.do_a2b_conf:
            self.gen_a2b_s = nn.ModuleList(self.gen_a2b_s)
            self.dis_a2b_s = nn.ModuleList(self.dis_a2b_s)
            if self.do_dis_council:
                self.dis_council_a2b_s = nn.ModuleList(self.dis_council_a2b_s)
        if self.do_b2a_conf:
            self.gen_b2a_s = nn.ModuleList(self.gen_b2a_s)
            self.dis_b2a_s = nn.ModuleList(self.dis_b2a_s)
            if self.do_dis_council:
                self.dis_council_b2a_s = nn.ModuleList(self.dis_council_b2a_s)

        # fix the noise used in sampling
        display_size = int(hyperparameters['display_size'])
        self.s_a = torch.randn(display_size, self.style_dim, 1, 1).cuda(self.cuda_device)
        self.s_b = torch.randn(display_size, self.style_dim, 1, 1).cuda(self.cuda_device)
        # Setup the optimizers
        beta1 = hyperparameters['beta1']
        beta2 = hyperparameters['beta2']
        dis_params_s = []
        gen_params_s = []
        self.dis_opt_s = []
        self.gen_opt_s = []
        self.dis_scheduler_s = []
        self.gen_scheduler_s = []
        if self.do_dis_council:
            dis_council_params_s = []
            self.dis_council_opt_s = []
            self.dis_council_scheduler_s = []
        for i in range(self.council_size):
            dis_parms = []
            gen_parms = []
            dis_council_parms = []
            if self.do_a2b_conf:
                dis_parms += list(self.dis_a2b_s[i].parameters())
                gen_parms += list(self.gen_a2b_s[i].parameters())
                if self.do_dis_council:
                    dis_council_parms += list(self.dis_council_a2b_s[i].parameters())
            if self.do_b2a_conf:
                dis_parms += list(self.dis_b2a_s[i].parameters())
                gen_parms += list(self.gen_b2a_s[i].parameters())
                if self.do_dis_council:
                    dis_council_parms += list(self.dis_council_b2a_s[i].parameters())
            dis_params_s.append(dis_parms)
            gen_params_s.append(gen_parms)
            if self.do_dis_council:
                dis_council_params_s.append(dis_council_parms)
            self.dis_opt_s.append(torch.optim.Adam([p for p in dis_params_s[i] if p.requires_grad],
                                                   lr=lr, betas=(beta1, beta2),
                                                   weight_decay=hyperparameters['weight_decay']))
            self.gen_opt_s.append(torch.optim.Adam([p for p in gen_params_s[i] if p.requires_grad],
                                                   lr=lr, betas=(beta1, beta2),
                                                   weight_decay=hyperparameters['weight_decay']))
            if self.do_dis_council:
                self.dis_council_opt_s.append(torch.optim.Adam([p for p in dis_council_params_s[i] if p.requires_grad],
                                                               lr=lr, betas=(beta1, beta2),
                                                               weight_decay=hyperparameters['weight_decay']))
            self.dis_scheduler_s.append(get_scheduler(self.dis_opt_s[i], hyperparameters))
            self.gen_scheduler_s.append(get_scheduler(self.gen_opt_s[i], hyperparameters))
            if self.do_dis_council:
                self.dis_council_scheduler_s.append(get_scheduler(self.dis_council_opt_s[i], hyperparameters))

        # Network weight initialization
        self.apply(weights_init(hyperparameters['init']))
        for i in range(self.council_size):
            if self.do_a2b_conf:
                self.gen_a2b_s[i].apply(weights_init(hyperparameters['init']))
                self.dis_a2b_s[i].apply(weights_init('gaussian'))
                if self.do_dis_council:
                    self.dis_council_a2b_s[i].apply(weights_init('gaussian'))
            if self.do_b2a_conf:
                self.gen_b2a_s[i].apply(weights_init(hyperparameters['init']))
                self.dis_b2a_s[i].apply(weights_init('gaussian'))
                if self.do_dis_council:
                    self.dis_council_b2a_s[i].apply(weights_init('gaussian'))

        # Load VGG model if needed
        self.vgg = None
        if 'vgg_w' in hyperparameters.keys() and hyperparameters['vgg_w'] > 0:
            self.vgg = load_vgg16(hyperparameters['vgg_model_path'] + '/models')
            self.vgg.eval()
            for param in self.vgg.parameters():
                param.requires_grad = False

    def recon_criterion(self, input, target):
        return torch.mean(torch.abs(input - target))

    def recon_criterion_v2_color(self, input, target):
        loss_1 = torch.mean(torch.abs(input - target))
        loss_2 = torch.mean(torch.pow((input - target), 2))
        if loss_1 > loss_2:
            return loss_1
        return loss_2

    def recon_criterion_v3_gray_scale(self, input, target):
        loss_1 = torch.mean(torch.abs(torch.sum(input, 1) - torch.sum(target, 1)))
        loss_2 = torch.mean(torch.pow((torch.sum(input, 1) - torch.sum(target, 1)), 2))
        if loss_1 > loss_2:
            return loss_1
        return loss_2

    def council_basic_criterion_gray_scale(self, input, target):
        return torch.mean(torch.abs(torch.sum(input, 1) - torch.sum(target, 1)))

    def council_basic_criterion_with_color(self, input, target):
        return torch.mean(torch.abs(input - target))

    def mask_zero_one_criterion(self, mask, center=0.5, epsilon=0.01):
        return torch.sum(1 / (torch.abs(mask - center) + epsilon)) / mask.numel()

    def mask_small_criterion(self, mask):
        assert self.hyperparameters['focus_loss']['mask_small_use_abs'] or self.hyperparameters['focus_loss']['mask_small_use_square'], 'at leas one small mask loss should be true, mask_small_use_abs or mask_small_use_square'
        loss = 0
        if self.hyperparameters['focus_loss']['mask_small_use_abs']:
            loss += self.mask_small_criterion_abs(mask)
        if self.hyperparameters['focus_loss']['mask_small_use_square']:
            loss += self.mask_small_criterion_square(mask)
        return loss

    def mask_small_criterion_square(self, mask):
        return (torch.sum(mask) / mask.numel()) ** 2

    def mask_small_criterion_abs(self, mask):
        return torch.abs((torch.sum(mask))) / mask.numel()

    def mask_criterion_TV(self, mask):
        return (torch.sum(torch.abs(mask[:, :, 1:, :]-mask[:, :, :-1, :])) + \
               torch.sum(torch.abs(mask[:, :, :, 1:] - mask[:, :, :, :-1]))) / mask.numel()

    def forward(self, x_a, x_b=None, s_a=None, s_b=None):
        self.eval()
        if self.do_a2b_conf:
            s_b = self.s_b if s_b is None else s_b
            s_b = Variable(s_b)
            x_ab_s = []
        if self.do_b2a_conf:
            s_a = self.s_a if s_a is None else s_a
            s_a = Variable(s_a)
            x_ba_s = []
        for i in range(self.council_size):
            if self.do_a2b_conf:
                c_a, s_a_fake = self.gen_a2b.encode(x_a)
                x_ab_s.append(self.gen_a2b.decode(c_a, s_b, x_a))
            if self.do_b2a_conf:
                c_b, s_b_fake = self.gen_b2a.encode(x_b)
                x_ba_s.append(self.gen_b2a.decode(c_b, s_a, x_b))

        self.train()
        if self.do_a2b_conf and self.do_b2a_conf:
            return x_ab_s, x_ba_s
        elif self.do_b2a_conf:
            return x_ba_s
        return x_ab_s

    def gen_update(self, x_a, x_b, hyperparameters, iterations=0):
        self.hyperparameters = hyperparameters
        for gen_opt in self.gen_opt_s:
            gen_opt.zero_grad()
        s_a = Variable(torch.randn(x_a.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
        s_b = Variable(torch.randn(x_b.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
        c_a_s = []
        s_a_prime_s = []
        c_b_s = []
        s_b_prime_s = []
        x_a_recon_s = []
        x_b_recon_s = []
        x_ba_s = []
        x_ab_s = []
        c_b_recon_s = []
        s_a_recon_s = []
        c_a_recon_s = []
        s_b_recon_s = []
        x_aba_s = []
        x_bab_s = []
        mask_ba_s = []
        mask_ab_s = []
        self.loss_gen_mask_zero_one_ba_s = []
        self.loss_gen_mask_zero_one_ab_s = []
        self.loss_gen_mask_total_ba_s = []
        self.loss_gen_mask_total_ab_s = []
        self.loss_gen_mask_TV_ab_s = []
        self.loss_gen_mask_TV_ba_s = []
        self.loss_gen_recon_x_a_s = []
        self.loss_gen_recon_x_b_s = []
        self.loss_gen_recon_s_a_s = []
        self.loss_gen_recon_s_b_s = []
        self.loss_gen_recon_c_a_s = []
        self.loss_gen_recon_c_b_s = []
        self.loss_gen_cycrecon_x_a_s = []
        self.loss_gen_cycrecon_x_b_s = []
        self.loss_gen_beginning_end_a_ab_s = []
        self.loss_gen_beginning_end_b_ba_s = []
        self.loss_gen_adv_a2b_s = []
        self.loss_gen_adv_b2a_s = []
        self.loss_gen_vgg_a_s = []
        self.loss_gen_vgg_b_s = []
        self.loss_gen_total_s = []
        self.council_w_conf = hyperparameters['council_w'] if hyperparameters['iteration'] > hyperparameters['council']['council_start_at_iter'] else 0
        self.mask_zero_or_one_w_conf = hyperparameters['mask_zero_or_one_w'] if hyperparameters['iteration'] > hyperparameters['focus_loss']['focus_loss_start_at_iter'] else 0
        self.mask_total_w_conf = hyperparameters['mask_total_w'] if hyperparameters['iteration'] > hyperparameters['focus_loss']['focus_loss_start_at_iter'] else 0
        self.mask_tv_w_conf = hyperparameters['mask_tv_w'] if hyperparameters['iteration'] > hyperparameters['focus_loss']['focus_loss_start_at_iter'] else 0

        for i in range(self.council_size):
            # encode
            if self.do_a2b_conf:
                c_a, s_a_prime = self.gen_a2b_s[i].encode(x_a)
                c_a_s.append(c_a)
                s_a_prime_s.append(s_a_prime)
            if self.do_b2a_conf:
                c_b, s_b_prime = self.gen_b2a_s[i].encode(x_b)
                c_b_s.append(c_b)
                s_b_prime_s.append(s_b_prime)

            # decode (within domain)
            if hyperparameters['recon_x_w'] != 0:
                if not self.do_a2b_conf and not self.do_b2a_conf:
                    print('cant do recon_x loss if not both do_a2b and b2a set to true')
                else:
                    x_a_recon_s.append(self.gen_b2a_s[i].decode(c_a_s[i], s_a_prime_s[i], x_a))
                    x_b_recon_s.append(self.gen_a2b_s[i].decode(c_b_s[i], s_b_prime_s[i], x_b))

            # decode (cross domain)
            if self.do_a2b_conf:
                x_ab_s.append(self.gen_a2b_s[i].decode(c_a_s[i], s_b, x_a))
            if self.do_b2a_conf:
                x_ba_s.append(self.gen_b2a_s[i].decode(c_b_s[i], s_a, x_b))

            if hyperparameters['mask_zero_or_one_w'] != 0 or hyperparameters['mask_total_w'] != 0:
                if self.do_a2b_conf:
                    mask_ab_s.append(self.gen_a2b_s[i].dec.mask_s)
                if self.do_b2a_conf:
                    mask_ba_s.append(self.gen_b2a_s[i].dec.mask_s)

            # encode again
            if hyperparameters['recon_s_w'] != 0 or hyperparameters['recon_c_w'] != 0 or hyperparameters['recon_x_cyc_w'] != 0:
                if not self.do_a2b_conf and not self.do_b2a_conf:
                    print('cant do recon_s and recon_c loss if not both do_a2b and b2a set to true')
                else:
                    c_b_recon, s_a_recon = self.gen_a2b_s[i].encode(x_ba_s[i])
                    c_a_recon, s_b_recon = self.gen_b2a_s[i].encode(x_ab_s[i])
                    c_b_recon_s.append(c_b_recon)
                    s_a_recon_s.append(s_a_recon)
                    c_a_recon_s.append(c_a_recon)
                    s_b_recon_s.append(s_b_recon)

            # decode again (if needed)
            if hyperparameters['recon_x_cyc_w'] != 0:
                if not self.do_a2b_conf and not self.do_b2a_conf:
                    print('cant do recon_x_cyc loss if not both do_a2b and b2a set to true')
                else:
                    x_aba_s.append(
                        self.gen_b2a.decode(c_a_recon_s[i], s_a_prime_s[i], x_a) if hyperparameters['recon_x_cyc_w'] > 0 else None)
                    x_bab_s.append(
                        self.gen_a2b.decode(c_b_recon_s[i], s_b_prime_s[i], x_b) if hyperparameters['recon_x_cyc_w'] > 0 else None)

            self.loss_gen_total_s.append(0)
            if hyperparameters['do_a2b']:
                self.loss_gen_mask_TV_ab_s.append(0)
                self.loss_gen_mask_total_ab_s.append(0)
            if hyperparameters['do_b2a']:
                self.loss_gen_mask_TV_ba_s.append(0)
                self.loss_gen_mask_total_ba_s.append(0)

            # masks should contain ones or zeros
            if hyperparameters['iteration'] > hyperparameters['focus_loss']['focus_loss_start_at_iter'] and (hyperparameters['mask_zero_or_one_w'] != 0 or hyperparameters['mask_total_w'] != 0 ):

                if hyperparameters['mask_zero_or_one_w'] != 0:
                    if hyperparameters['do_a2b']:
                        self.loss_gen_mask_zero_one_ab_s.append(self.mask_zero_one_criterion(mask_ab_s[i], center=hyperparameters['focus_loss']['mask_zero_or_one_center'], epsilon=hyperparameters['focus_loss']['mask_zero_or_one_epsilon']))
                    if hyperparameters['do_b2a']:
                        self.loss_gen_mask_zero_one_ba_s.append(self.mask_zero_one_criterion(mask_ba_s[i], center=hyperparameters['focus_loss']['mask_zero_or_one_center'], epsilon=hyperparameters['focus_loss']['mask_zero_or_one_epsilon']))

                    if self.do_w_loss_matching_focus:
                        if hyperparameters['do_a2b']:
                            self.los_hist_focus_zero_one_a2b_s[i].append(self.loss_gen_mask_zero_one_ab_s[i].detach().cpu().numpy())
                            self.los_hist_focus_zero_one_a2b_s[i].popleft()
                            self.w_match_focus_zero_one_a2b_conf = np.mean(self.los_hist_gan_a2b_s[i]) / np.mean(self.los_hist_focus_zero_one_a2b_s[i])
                            self.loss_gen_mask_zero_one_ab_s[i] *= self.w_match_focus_zero_one_a2b_conf
                            self.loss_gen_total_s[i] += hyperparameters['mask_zero_or_one_w'] * self.loss_gen_mask_zero_one_ab_s[i].cuda(self.cuda_device)
                        if hyperparameters['do_b2a']:
                            self.los_hist_focus_zero_one_b2a_s[i].append(self.loss_gen_mask_zero_one_ba_s[i].detach().cpu().numpy())
                            self.los_hist_focus_zero_one_b2a_s[i].popleft()
                            self.w_match_focus_zero_one_b2a_conf = np.mean(self.los_hist_gan_b2a_s[i]) / np.mean(self.los_hist_focus_zero_one_b2a_s[i])
                            self.loss_gen_mask_zero_one_ba_s[i] *= self.w_match_focus_zero_one_b2a_conf
                            self.loss_gen_total_s[i] += hyperparameters['mask_zero_or_one_w'] * self.loss_gen_mask_zero_one_ba_s[i].cuda(self.cuda_device)
                    else:
                        if hyperparameters['do_a2b']:
                            self.loss_gen_total_s[i] += hyperparameters['mask_zero_or_one_w'] * self.loss_gen_mask_zero_one_ab_s[i].cuda(self.cuda_device)
                        if hyperparameters['do_b2a']:
                            self.loss_gen_total_s[i] += hyperparameters['mask_zero_or_one_w'] * self.loss_gen_mask_zero_one_ba_s[i].cuda(self.cuda_device)

                # masks should as small as possible to leave original domain with little changes
                if hyperparameters['mask_total_w'] != 0:
                    if hyperparameters['do_a2b']:
                        self.loss_gen_mask_total_ab_s[i] += self.mask_small_criterion(mask_ab_s[i])
                    if hyperparameters['do_b2a']:
                        self.loss_gen_mask_total_ba_s[i] += self.mask_small_criterion(mask_ba_s[i])

                # TV loss on the mask
                if hyperparameters['mask_tv_w'] != 0:
                    if hyperparameters['do_a2b']:
                        self.loss_gen_mask_TV_ab_s[i] += self.mask_criterion_TV(mask_ab_s[i])
                        self.loss_gen_total_s[i] += hyperparameters['mask_tv_w'] * self.loss_gen_mask_TV_ab_s[i]
                    if hyperparameters['do_b2a']:
                        self.loss_gen_mask_TV_ba_s[i] += self.mask_criterion_TV(mask_ba_s[i])
                        self.loss_gen_total_s[i] += hyperparameters['mask_tv_w'] * self.loss_gen_mask_TV_ba_s[i]

                if self.do_w_loss_matching_focus:
                    if hyperparameters['do_a2b']:
                        self.los_hist_focus_a2b_s[i].append(self.loss_gen_mask_total_ab_s[i].detach().cpu().numpy())
                        self.los_hist_focus_a2b_s[i].popleft()
                        self.w_match_focus_a2b_conf = np.mean(self.los_hist_gan_a2b_s[i]) / np.mean(self.los_hist_focus_a2b_s[i])
                        self.loss_gen_mask_total_ab_s[i] *= self.w_match_focus_a2b_conf
                        self.loss_gen_total_s[i] += hyperparameters['mask_total_w'] * self.loss_gen_mask_total_ab_s[i].cuda(self.cuda_device)
                    if hyperparameters['do_b2a']:
                        self.los_hist_focus_b2a_s[i].append(self.loss_gen_mask_total_ab_s[i].detach().cpu().numpy())
                        self.los_hist_focus_b2a_s[i].popleft()
                        self.w_match_focus_b2a_conf = np.mean(self.los_hist_gan_b2a_s[i]) / np.mean(self.los_hist_focus_b2a_s[i])
                        self.loss_gen_mask_total_ba_s[i] *= self.w_match_focus_b2a_conf
                        self.loss_gen_total_s[i] += hyperparameters['mask_total_w'] * self.loss_gen_mask_total_ba_s[i].cuda(self.cuda_device)

                else:
                    if hyperparameters['do_a2b']:
                        self.loss_gen_total_s[i] += hyperparameters['mask_total_w'] * self.loss_gen_mask_total_ab_s[i].cuda(self.cuda_device)
                    if hyperparameters['do_b2a']:
                        self.loss_gen_total_s[i] += hyperparameters['mask_total_w'] * self.loss_gen_mask_total_ba_s[i].cuda(self.cuda_device)


            # reconstruction loss
            if hyperparameters['recon_x_w'] != 0 and self.do_a2b_conf and self.do_b2a_conf:
                self.loss_gen_recon_x_a_s.append(self.recon_criterion(x_a_recon_s[i], x_a))
                self.loss_gen_recon_x_b_s.append(self.recon_criterion(x_b_recon_s[i], x_b))
                self.loss_gen_total_s[i] += hyperparameters['recon_x_w'] * (
                        self.loss_gen_recon_x_a_s[i].cuda(self.cuda_device) + self.loss_gen_recon_x_b_s[i].cuda(self.cuda_device))
            if hyperparameters['recon_s_w'] != 0 and self.do_a2b_conf and self.do_b2a_conf:
                self.loss_gen_recon_s_a_s.append(self.recon_criterion(s_a_recon_s[i], s_a))
                self.loss_gen_recon_s_b_s.append(self.recon_criterion(s_b_recon_s[i], s_b))
                self.loss_gen_total_s[i] += hyperparameters['recon_s_w'] * (
                        self.loss_gen_recon_s_a_s[i].cuda(self.cuda_device) + self.loss_gen_recon_s_b_s[i].cuda(self.cuda_device))
            if hyperparameters['recon_c_w'] != 0 and self.do_a2b_conf and self.do_b2a_conf:
                self.loss_gen_recon_c_a_s.append(self.recon_criterion(c_a_recon_s[i], c_a_s[i]))
                self.loss_gen_recon_c_b_s.append(self.recon_criterion(c_b_recon_s[i], c_b_s[i]))
                self.loss_gen_total_s[i] += hyperparameters['recon_c_w'] * (
                        self.loss_gen_recon_c_a_s[i].cuda(self.cuda_device) + self.loss_gen_recon_c_b_s[i].cuda(self.cuda_device))
            if hyperparameters['recon_x_cyc_w'] != 0 and self.do_a2b_conf and self.do_b2a_conf:
                self.loss_gen_cycrecon_x_a_s.append(
                    self.recon_criterion(x_aba_s[i], x_a) if hyperparameters['recon_x_cyc_w'] > 0 else 0)
                self.loss_gen_cycrecon_x_b_s.append(
                    self.recon_criterion(x_bab_s[i], x_b) if hyperparameters['recon_x_cyc_w'] > 0 else 0)
                self.loss_gen_total_s[i] += hyperparameters['recon_x_cyc_w'] * (
                        self.loss_gen_cycrecon_x_a_s[i].cuda(self.cuda_device) + self.loss_gen_cycrecon_x_b_s[i].cuda(self.cuda_device))
            if hyperparameters['abs_beginning_end'] != 0 and self.abs_beginning_end_w_conf > 0.005:
                if hyperparameters['do_a2b']:
                    self.loss_gen_beginning_end_a_ab_s.append(
                        self.recon_criterion_v2_color(x_ab_s[i], x_a) if hyperparameters['abs_beginning_end'] > 0 or hyperparameters['abs_beginning_end_minimume'] > 0 else 0)
                else:
                    self.loss_gen_beginning_end_a_ab_s.append(0)
                if hyperparameters['do_b2a']:
                    self.loss_gen_beginning_end_b_ba_s.append(
                        self.recon_criterion_v2_color(x_ba_s[i], x_b) if hyperparameters['abs_beginning_end'] > 0 or hyperparameters['abs_beginning_end_minimume'] > 0 else 0)
                else:
                    self.loss_gen_beginning_end_b_ba_s.append(0)

                self.abs_beginning_end_w_conf = hyperparameters['abs_beginning_end'] * (hyperparameters['abs_beginning_end_less_by'] ** iterations)
                self.abs_beginning_end_w_conf = max(self.abs_beginning_end_w_conf, hyperparameters['abs_beginning_end_minimume'])

                if hyperparameters['do_a2b']:
                    self.loss_gen_total_s[i] += self.abs_beginning_end_w_conf * self.loss_gen_beginning_end_a_ab_s[i].cuda(self.cuda_device)
                if hyperparameters['do_b2a']:
                    self.loss_gen_total_s[i] += self.abs_beginning_end_w_conf * self.loss_gen_beginning_end_b_ba_s[i].cuda(self.cuda_device)

            # GAN loss
            if hyperparameters['gan_w'] != 0:
                i_dis = i
                if hyperparameters['gen']['useRandomDis']:
                    i_dis = np.random.randint(self.council_size)

                if hyperparameters['do_a2b']:
                    x_ab_s_curr = x_ab_s[i] if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_ab_s[i], 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_b'], 1, 1) / hyperparameters['input_dim_b']
                    loss_gen_adv_a2b = self.dis_a2b_s[i_dis].calc_gen_loss(x_ab_s_curr)
                else:
                    loss_gen_adv_a2b = 0

                if hyperparameters['do_b2a']:
                    x_ba_s_curr = x_ba_s[i] if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_ba_s[i], 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_a'], 1, 1) / hyperparameters['input_dim_a']
                    loss_gen_adv_b2a = self.dis_b2a_s[i_dis].calc_gen_loss(x_ba_s_curr)
                else:
                    loss_gen_adv_b2a = 0

                self.loss_gen_adv_a2b_s.append(loss_gen_adv_a2b)
                self.loss_gen_adv_b2a_s.append(loss_gen_adv_b2a)

                if self.do_w_loss_matching:
                    if hyperparameters['do_a2b']:
                        self.los_hist_gan_a2b_s[i].append(loss_gen_adv_a2b.detach().cpu().numpy())
                        self.los_hist_gan_a2b_s[i].popleft()
                    if hyperparameters['do_b2a']:
                        self.los_hist_gan_b2a_s[i].append(loss_gen_adv_b2a.detach().cpu().numpy())
                        self.los_hist_gan_b2a_s[i].popleft()

                if hyperparameters['do_a2b']:
                    self.loss_gen_total_s[i] += hyperparameters['gan_w'] * self.loss_gen_adv_a2b_s[i].cuda(self.cuda_device)
                if hyperparameters['do_b2a']:
                    self.loss_gen_total_s[i] += hyperparameters['gan_w'] * self.loss_gen_adv_b2a_s[i].cuda(self.cuda_device)

            # domain-invariant perceptual loss
            if hyperparameters['vgg_w'] != 0:
                self.loss_gen_vgg_a_s.append(
                    self.compute_vgg_loss(self.vgg, x_ba_s[i], x_b) if hyperparameters['vgg_w'] > 0 else 0)
                self.loss_gen_vgg_b_s.append(
                    self.compute_vgg_loss(self.vgg, x_ab_s[i], x_a) if hyperparameters['vgg_w'] > 0 else 0)
                self.loss_gen_total_s[i] += hyperparameters['vgg_w'] * (
                        self.loss_gen_vgg_a_s[i].cuda(self.cuda_device) + self.loss_gen_vgg_b_s[i].cuda(self.cuda_device))

        # Council loss
        onOffCycle = hyperparameters['council']['flipOnOff_On_iteration'] + hyperparameters['council'][
            'flipOnOff_Off_iteration']
        currIterCyc = hyperparameters['iteration'] % onOffCycle
        if hyperparameters['council']['flipOnOff_start_with']:
            startCyc = hyperparameters['council']['flipOnOff_On_iteration']
        else:
            startCyc = hyperparameters['council']['flipOnOff_Off_iteration']

        self.do_council_loss = hyperparameters['council']['flipOnOff_start_with'] if (currIterCyc < startCyc) \
            else not hyperparameters['council']['flipOnOff_start_with']

        if not hyperparameters['council']['flipOnOff']:
            self.do_council_loss = True
        if hyperparameters['iteration'] < hyperparameters['council']['council_start_at_iter']:
            self.do_council_loss = False
        self.council_loss_ba_s = []
        self.council_loss_ab_s = []
        for i in range(self.council_size):
            if (hyperparameters['council_w'] != 0 or hyperparameters['council_abs_w'] != 0) and self.do_council_loss and self.council_size > 1:
                # if i == 0:
                #     print('do council loss: True')
                if self.do_a2b_conf:
                    self.council_loss_ab_s.append(0)
                if self.do_b2a_conf:
                    self.council_loss_ba_s.append(0)

                if self.do_dis_council:  # do council discriminator
                    if hyperparameters['do_a2b']:
                        dis_council_loss_ab = self.dis_council_a2b_s[i].calc_gen_loss(x_ab_s[i], x_a)
                    else:
                        dis_council_loss_ab = 0
                    if hyperparameters['do_b2a']:
                        dis_council_loss_ba = self.dis_council_b2a_s[i].calc_gen_loss(x_ba_s[i], x_b)
                    else:
                        dis_council_loss_ba = 0
                    if self.do_w_loss_matching:
                        if hyperparameters['do_a2b']:
                            self.los_hist_council_a2b_s[i].append(dis_council_loss_ab.detach().cpu().numpy())
                            self.los_hist_council_a2b_s[i].popleft()
                            self.w_match_a2b_conf = np.mean(self.los_hist_gan_a2b_s[i]) / np.mean(self.los_hist_council_a2b_s[i])
                            dis_council_loss_ab *= self.w_match_a2b_conf
                        if hyperparameters['do_b2a']:
                            self.los_hist_council_b2a_s[i].append(dis_council_loss_ba.detach().cpu().numpy())
                            self.los_hist_council_b2a_s[i].popleft()
                            self.w_match_b2a_conf = np.mean(self.los_hist_gan_b2a_s[i]) / np.mean(self.los_hist_council_b2a_s[i])
                            dis_council_loss_ba *= self.w_match_b2a_conf

                    if hyperparameters['do_a2b']:
                        dis_council_loss_ab *= hyperparameters['council_w']
                        self.council_loss_ab_s[i] += dis_council_loss_ab
                    if hyperparameters['do_b2a']:
                        dis_council_loss_ba *= hyperparameters['council_w']
                        self.council_loss_ba_s[i] += dis_council_loss_ba

                if hyperparameters['council_abs_w'] != 0 and self.council_size > 1:  # ads loss without discriminetor
                    tmp = list(range(0, i)) + list(range(i + 1, self.council_size))
                    comper_to_i = random.choice(tmp)
                    if hyperparameters['council_abs_gray_scale']:
                        if hyperparameters['do_a2b']:
                            abs_council_loss_ab = hyperparameters['council_abs_w'] * self.council_basic_criterion_gray_scale(x_ab_s[i], x_ab_s[comper_to_i].detach())
                        else:
                            abs_council_loss_ab = 0
                        if hyperparameters['do_b2a']:
                            abs_council_loss_ba = hyperparameters['council_abs_w'] * self.council_basic_criterion_gray_scale(x_ba_s[i], x_ba_s[comper_to_i].detach())
                        else:
                            abs_council_loss_ba = 0
                    else:
                        if hyperparameters['do_a2b']:
                            abs_council_loss_ab = hyperparameters['council_abs_w'] * self.council_basic_criterion_with_color(x_ab_s[i], x_ab_s[comper_to_i].detach())
                        else:
                            abs_council_loss_ab = 0
                        if hyperparameters['do_b2a']:
                            abs_council_loss_ba = hyperparameters['council_abs_w'] * self.council_basic_criterion_with_color(x_ba_s[i], x_ba_s[comper_to_i].detach())
                        else:
                            abs_council_loss_ba = 0
                    if self.do_a2b_conf:
                        self.council_loss_ab_s[i] += abs_council_loss_ba.cuda(self.cuda_device)
                    if self.do_b2a_conf:
                        self.council_loss_ba_s[i] += abs_council_loss_ab.cuda(self.cuda_device)

                if hyperparameters['do_a2b']:
                    self.loss_gen_total_s[i] += self.council_loss_ab_s[i].cuda(self.cuda_device)
                if hyperparameters['do_b2a']:
                        self.loss_gen_total_s[i] += self.council_loss_ba_s[i].cuda(self.cuda_device)

            else:
                if self.do_a2b_conf:
                    self.council_loss_ab_s.append(0)
                if self.do_b2a_conf:
                    self.council_loss_ba_s.append(0)

            # backpropogation
            self.loss_gen_total_s[i].backward()
            self.gen_opt_s[i].step()

    def compute_vgg_loss(self, vgg, img, target):
        img_vgg = vgg_preprocess(img)
        target_vgg = vgg_preprocess(target)
        img_fea = vgg(img_vgg)
        target_fea = vgg(target_vgg)
        return torch.mean((self.instancenorm(img_fea) - self.instancenorm(target_fea)) ** 2)

    def sample(self, x_a=None, x_b=None, s_a=None, s_b=None, council_member_to_sample_vec=None, return_mask=True):
        self.eval()
        if self.do_a2b_conf:
            x_a_s = []
            s_b = self.s_b if s_b is None else s_b
            s_b1 = Variable(s_b)
            s_b2 = Variable(torch.randn(x_a.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
            x_a_recon, x_ab1, x_ab2, x_ab1_mask = [], [], [], []
        if self.do_b2a_conf:
            x_b_s = []
            s_a = self.s_a if s_a is None else s_a
            s_a1 = Variable(s_a)
            s_a2 = Variable(torch.randn(x_b.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
            x_b_recon, x_ba1, x_ba2, x_ba1_mask = [], [], [], []

        council_member_to_sample_vec = range(self.council_size) if council_member_to_sample_vec is None else council_member_to_sample_vec
        x_size = x_a.size(0) if x_a is not None else x_b.size(0)
        for i in range(x_size):
            for j in council_member_to_sample_vec:
                if self.do_b2a_conf:
                    x_b_s.append(x_b[i].unsqueeze(0))
                    c_b, s_b_fake = self.gen_b2a_s[j].encode(x_b[i].unsqueeze(0))
                    if not return_mask:
                        x_b_recon.append(self.gen_b2a_s[j].decode(c_b, s_b_fake, x_b[i].unsqueeze(0)))
                        x_ba1.append(self.gen_b2a_s[j].decode(c_b, s_a1[i].unsqueeze(0), x_b[i].unsqueeze(0)))
                        x_ba2.append(self.gen_b2a_s[j].decode(c_b, s_a2[i].unsqueeze(0), x_b[i].unsqueeze(0)))
                    else:
                        x_ba1_tmp, x_ba1_mask_tmp = self.gen_b2a_s[j].decode(c_b, s_a1[i].unsqueeze(0), x_b[i].unsqueeze(0), return_mask=return_mask)
                        x_ba1_mask.append(x_ba1_mask_tmp)
                        x_ba1.append(x_ba1_tmp)
                        x_ba2.append(self.gen_b2a_s[j].decode(c_b, s_a2[i].unsqueeze(0), x_b[i].unsqueeze(0)))
                if self.do_a2b_conf:
                    x_a_s.append(x_a[i].unsqueeze(0))
                    c_a, s_a_fake = self.gen_a2b_s[j].encode(x_a[i].unsqueeze(0))
                    if not return_mask:
                        x_a_recon.append(self.gen_a2b_s[j].decode(c_a, s_a_fake, x_a[i].unsqueeze(0)))
                        x_ab1.append(self.gen_a2b_s[j].decode(c_a, s_b1[i].unsqueeze(0), x_a[i].unsqueeze(0)))
                        x_ab2.append(self.gen_a2b_s[j].decode(c_a, s_b2[i].unsqueeze(0), x_a[i].unsqueeze(0)))
                    else:
                        x_ab1_tmp, x_ab1_mask_tmp = self.gen_a2b_s[j].decode(c_a, s_b1[i].unsqueeze(0), x_a[i].unsqueeze(0), return_mask=return_mask)
                        do_double = False
                        if do_double:
                            c_a_double, s_a_fake = self.gen_a2b_s[j].encode(x_ab1_tmp)
                            x_ab1_tmp, x_ab1_mask_tmp = self.gen_a2b_s[j].decode(c_a_double, s_b1[i].unsqueeze(0),
                                                                               x_ab1_tmp,
                                                                               return_mask=return_mask)

                        x_ab1_mask.append(x_ab1_mask_tmp)
                        x_ab1.append(x_ab1_tmp)
                        x_ab2.append(self.gen_a2b_s[j].decode(c_a, s_b2[i].unsqueeze(0), x_a[i].unsqueeze(0)))

        if self.do_b2a_conf:
            x_b_s = torch.cat(x_b_s)
            x_ba1, x_ba2 = torch.cat(x_ba1), torch.cat(x_ba2)
            if not return_mask:
                x_b_recon = torch.cat(x_b_recon)
            else:
                x_ba1_mask = torch.cat(x_ba1_mask)
        if self.do_a2b_conf:
            x_a_s = torch.cat(x_a_s)
            x_ab1, x_ab2 = torch.cat(x_ab1), torch.cat(x_ab2)
            if not return_mask:
                x_a_recon = torch.cat(x_a_recon)
            else:
                x_ab1_mask = torch.cat(x_ab1_mask)

        self.train()

        do_diff = False
        if do_diff:
            if self.do_a2b_conf:
                x_ab1 = x_a_s - x_ab1
                x_ab2 = x_a_s - x_ab2
            if self.do_b2a_conf:
                x_ba1 = x_b_s - x_ba1
                x_ba2 = x_b_s - x_ba2

        if not return_mask:
            if self.do_a2b_conf and self.do_b2a_conf:
                return x_a_s, x_a_recon, x_ab1, x_ab2, x_b_s, x_b_recon, x_ba1, x_ba2
            if self.do_a2b_conf:
                return x_a_s, x_a_recon, x_ab1, x_ab2, None, None, None, None
            if self.do_b2a_conf:
                return None, None, None, None, x_b_s, x_b_recon, x_ba1, x_ba2
        else:
            if self.do_a2b_conf and self.do_b2a_conf:
                return x_a_s, x_ab1_mask, x_ab1, x_ab2, x_b_s, x_ba1_mask, x_ba1, x_ba2
            if self.do_a2b_conf:
                return x_a_s, x_ab1_mask, x_ab1, x_ab2, None, None, None, None
            if self.do_b2a_conf:
                return None, None, None, None, x_b_s, x_ba1_mask, x_ba1, x_ba2

    def dis_update(self, x_a=None, x_b=None, hyperparameters=None):
        x_a_dis = x_a if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_a.detach(), 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_a'], 1, 1) / hyperparameters['input_dim_a']
        x_b_dis = x_b if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_b.detach(), 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_b'], 1, 1) / hyperparameters['input_dim_b']
        for dis_opt in self.dis_opt_s:
            dis_opt.zero_grad()
        if self.do_a2b_conf:
            s_b = Variable(torch.randn(x_b.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
            self.loss_dis_a2b_s = []
        if self.do_b2a_conf:
            s_a = Variable(torch.randn(x_a.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
            self.loss_dis_b2a_s = []
        self.loss_dis_total_s = []
        for i in range(self.council_size):
            i_gen = i
            if hyperparameters['dis']['useRandomGen']:
                i_gen = np.random.randint(self.council_size)

            # encode
            if hyperparameters['do_a2b']:
                c_a, _ = self.gen_a2b_s[i_gen].encode(x_a)
            if hyperparameters['do_b2a']:
                c_b, _ = self.gen_b2a_s[i_gen].encode(x_b)

            # decode (cross domain)
            if hyperparameters['do_a2b']:
                x_ab = self.gen_a2b_s[i_gen].decode(c_a, s_b, x_a)
                x_ab = x_ab if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_ab.detach(), 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_b'], 1, 1) / hyperparameters['input_dim_b']

            if hyperparameters['do_b2a']:
                x_ba = self.gen_b2a_s[i_gen].decode(c_b, s_a, x_b)
                x_ba = x_ba if not hyperparameters['dis']['do_Dis_only_gray'] else torch.sum(x_ba.detach(), 1).unsqueeze(1).repeat(1, hyperparameters['input_dim_a'], 1, 1) / hyperparameters['input_dim_a']

            # D loss
            if hyperparameters['do_a2b']:
                self.loss_dis_a2b_s.append(self.dis_a2b_s[i].calc_dis_loss(x_ab.detach(), x_b_dis))
            if hyperparameters['do_b2a']:
                self.loss_dis_b2a_s.append(self.dis_b2a_s[i].calc_dis_loss(x_ba.detach(), x_a_dis))

            self.loss_dis_total_s.append(0)
            if hyperparameters['do_a2b']:
                self.loss_dis_total_s[i] += hyperparameters['gan_w'] * self.loss_dis_a2b_s[i]
            if hyperparameters['do_b2a']:
                self.loss_dis_total_s[i] += self.loss_dis_b2a_s[i]

            self.loss_dis_total_s[i].backward()
            self.dis_opt_s[i].step()

    def dis_council_update(self, x_a=None, x_b=None, hyperparameters=None):

        if self.council_size <= 1 or hyperparameters['council']['numberOfCouncil_dis_relative_iteration'] == 0:
            print('no council discriminetor is needed (council size <= 1 or numberOfCouncil_dis_relative_iteration == 0)')
            return
        onOffCycle = hyperparameters['council']['flipOnOff_On_iteration'] + hyperparameters['council'][
            'flipOnOff_Off_iteration']
        currIterCyc = hyperparameters['iteration'] % onOffCycle
        if hyperparameters['council']['flipOnOff_start_with']:
            startCyc = hyperparameters['council']['flipOnOff_On_iteration']
        else:
            startCyc = hyperparameters['council']['flipOnOff_Off_iteration']

        self.do_council_loss = hyperparameters['council']['flipOnOff_start_with'] if (currIterCyc < startCyc) \
            else not hyperparameters['council']['flipOnOff_start_with']
        if not hyperparameters['council']['flipOnOff']:
            self.do_council_loss = hyperparameters['council']['flipOnOff_start_with']

        if not self.do_council_loss or hyperparameters['council_w'] == 0 or hyperparameters['iteration'] < hyperparameters['council']['council_start_at_iter']:
            return

        for dis_council_opt in self.dis_council_opt_s:
            dis_council_opt.zero_grad()

        if self.do_b2a_conf:
            s_a = Variable(torch.randn(x_a.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
        if self.do_a2b_conf:
            s_b = Variable(torch.randn(x_b.size(0), self.style_dim, 1, 1).cuda(self.cuda_device))
        if hyperparameters['council']['discriminetro_less_style_by'] != 0:
            if self.do_b2a_conf:
                s_a_less = s_a * hyperparameters['council']['discriminetro_less_style_by']
            if self.do_a2b_conf:
                s_b_less = s_b * hyperparameters['council']['discriminetro_less_style_by']

        self.loss_dis_council_a2b_s = []
        self.loss_dis_council_b2a_s = []
        self.loss_dis_council_total_s = []
        c_a_s = []
        c_b_s = []
        x_ba_s = []
        x_ab_s = []
        x_ba_s_less = []
        x_ab_s_less = []

        for i in range(self.council_size):
            # encode
            if hyperparameters['do_a2b']:
                c_a, _ = self.gen_a2b_s[i].encode(x_a)
                c_a_s.append(c_a)
            if hyperparameters['do_b2a']:
                c_b, _ = self.gen_b2a_s[i].encode(x_b)
                c_b_s.append(c_b)

            # decode (cross domain)
            if hyperparameters['do_a2b']:
                x_ab = self.gen_a2b_s[i].decode(c_a, s_b, x_a)
                x_ab_s.append(x_ab)
            if hyperparameters['do_b2a']:
                x_ba = self.gen_b2a_s[i].decode(c_b, s_a, x_b)
                x_ba_s.append(x_ba)

            if hyperparameters['council']['discriminetro_less_style_by'] != 0:
                # decode (cross domain) less_style_by
                if hyperparameters['do_a2b']:
                    x_ab_less = self.gen_a2b_s[i].decode(c_a, s_b_less, x_a)
                    x_ab_s_less.append(x_ab_less)

                if hyperparameters['do_b2a']:
                    x_ba_less = self.gen_b2a_s[i].decode(c_b, s_a_less, x_b)
                    x_ba_s_less.append(x_ba_less)

        if self.do_a2b_conf:
            comper_x_ab_s = x_ab_s if hyperparameters['council']['discriminetro_less_style_by'] == 0 else x_ab_s_less
        if self.do_b2a_conf:
            comper_x_ba_s = x_ba_s if hyperparameters['council']['discriminetro_less_style_by'] == 0 else x_ba_s_less

        for i in range(self.council_size):
            self.loss_dis_council_a2b_s.append(0)
            self.loss_dis_council_b2a_s.append(0)
            index_to_chose_from = list(range(0, i)) + list(range(i + 1, self.council_size))
            for k in range(hyperparameters['council']['numberOfCouncil_dis_relative_iteration']):
                if k == self.council_size:
                    break
                if len(index_to_chose_from) == 0:
                    index_to_chose_from = list(range(0, i)) + list(range(i + 1, self.council_size)) # reinitilize the indexes to chose from if numberOfCouncil_dis_relative_iteration is biger then thr number of council members
                index_to_comper = random.choice(index_to_chose_from)
                index_to_chose_from.remove(index_to_comper)

                # D loss
                if hyperparameters['do_a2b']:
                    self.loss_dis_council_a2b_s[i] += self.dis_council_a2b_s[i].calc_dis_loss(x_ab_s[i].detach(), comper_x_ab_s[index_to_comper].detach(), x_a)  # original
                if hyperparameters['do_b2a']:
                    self.loss_dis_council_b2a_s[i] += self.dis_council_b2a_s[i].calc_dis_loss(x_ba_s[i].detach(), comper_x_ba_s[index_to_comper].detach(), x_b)  # original

            self.loss_dis_council_total_s.append(0)
            if hyperparameters['do_a2b']:
                self.loss_dis_council_total_s[i] += hyperparameters['council_w'] * self.loss_dis_council_a2b_s[i] / hyperparameters['council']['numberOfCouncil_dis_relative_iteration']
            if hyperparameters['do_b2a']:
                self.loss_dis_council_total_s[i] += hyperparameters['council_w'] * self.loss_dis_council_b2a_s[i] / hyperparameters['council']['numberOfCouncil_dis_relative_iteration']

            self.loss_dis_council_total_s[i].backward()
            self.dis_council_opt_s[i].step()

    def update_learning_rate(self):
        for dis_scheduler in self.dis_scheduler_s:
            if dis_scheduler is not None:
                dis_scheduler.step()
        for gen_scheduler in self.gen_scheduler_s:
            if gen_scheduler is not None:
                gen_scheduler.step()
        if not self.do_dis_council:
            return
        for dis_council_scheduler in self.dis_council_scheduler_s:
            if dis_council_scheduler is not None:
                dis_council_scheduler.step()

    def resume(self, checkpoint_dir, hyperparameters):
        iterations = 0
        # Load generators
        for i in range(self.council_size):
            last_model_name = get_model_list(checkpoint_dir, "gen_" + str(i))
            if last_model_name is not None:
                last_model_name = last_model_name.replace('a2b_gen_', 'gen_').replace('b2a_gen_', 'gen_')
                print('loading: ' + last_model_name)
                if self.do_a2b_conf:
                    state_dict = torch.load(last_model_name.replace('gen_', 'a2b_gen_'), map_location=torch.device(self.cuda_device))
                    self.gen_a2b_s[i].load_state_dict(state_dict['a2b'])
                if self.do_b2a_conf:
                    state_dict = torch.load(last_model_name.replace('gen_', 'b2a_gen_'), map_location=torch.device(self.cuda_device))
                    self.gen_b2a_s[i].load_state_dict(state_dict['b2a'])
                iterations = int(last_model_name[-11:-3])
            else:
                warnings.warn('Failed to find gen checkpoint, did not load model')

            # Load discriminators
            last_model_name = get_model_list(checkpoint_dir, "dis_" + str(i))
            if last_model_name is not None:
                last_model_name = last_model_name.replace('a2b_dis_', 'dis_').replace('b2a_dis_', 'dis_')
                print('loading: ' + last_model_name)
                if self.do_a2b_conf:
                    state_dict = torch.load(last_model_name.replace('dis_', 'a2b_dis_'), map_location=torch.device(self.cuda_device))
                    self.dis_a2b_s[i].load_state_dict(state_dict['a2b'])
                if self.do_b2a_conf:
                    state_dict = torch.load(last_model_name.replace('dis_', 'b2a_dis_'), map_location=torch.device(self.cuda_device))
                    self.dis_b2a_s[i].load_state_dict(state_dict['b2a'])
            else:
                warnings.warn('Failed to find dis checkpoint, did not load model')
            # Load council discriminators
            if self.do_dis_council:
                try:
                    last_model_name = get_model_list(checkpoint_dir, "dis_council_" + str(i))
                    print('loading: ' + last_model_name)
                    if last_model_name is not None:
                        last_model_name = last_model_name.replace('a2b_dis_council_', 'dis_council_').replace('b2a_dis_council_', 'dis_council_')

                        if self.do_a2b_conf:
                            state_dict = torch.load(last_model_name.replace('dis_council_', 'a2b_dis_council_'), map_location=torch.device(self.cuda_device))
                            self.dis_council_a2b_s[i].load_state_dict(state_dict['a2b'])
                        if self.do_b2a_conf:
                            state_dict = torch.load(last_model_name.replace('dis_council_', 'b2a_dis_council_'), map_location=torch.device(self.cuda_device))
                            self.dis_council_b2a_s[i].load_state_dict(state_dict['b2a'])
                    else:
                        warnings.warn('Failed to find dis checkpoint, did not load model')
                except:
                    warnings.warn('some council discriminetor FAILED to load')

            # Load optimizers
            try:
                state_dict = torch.load(os.path.join(checkpoint_dir, 'optimizer_' + str(i) + '.pt'), map_location=torch.device(self.cuda_device))
                self.dis_opt_s[i].load_state_dict(state_dict['dis'])
                self.gen_opt_s[i].load_state_dict(state_dict['gen'])
                if self.do_dis_council:
                    self.dis_council_opt_s[i].load_state_dict(state_dict['dis_council'])

                # Reinitilize schedulers
                self.dis_scheduler_s[i] = get_scheduler(self.dis_opt_s[i], hyperparameters, iterations)
                self.gen_scheduler = get_scheduler(self.gen_opt_s[i], hyperparameters, iterations)
                if self.do_dis_council:
                    self.dis_council_scheduler_s[i] = get_scheduler(self.dis_council_opt_s[i], hyperparameters, iterations)
            except:
                warnings.warn('some optimizer FAILED to load ')
        if iterations > 0 :
            print('Resume from iteration %d' % iterations)
        else:
            warnings.warn('FAILED TO RESUME STARTED FROM 0')
        return iterations

    def save(self, snapshot_dir, iterations):
        for i in range(self.council_size):

            # Save generators, discriminators, and optimizers
            gen_name = os.path.join(snapshot_dir, 'gen_' + str(i) + '_%08d.pt' % (iterations + 1))
            dis_name = os.path.join(snapshot_dir, 'dis_' + str(i) + '_%08d.pt' % (iterations + 1))
            if self.do_dis_council:
                dis_council_name = os.path.join(snapshot_dir, 'dis_council_' + str(i) + '_%08d.pt' % (iterations + 1))
            opt_name = os.path.join(snapshot_dir, 'optimizer_' + str(i) + '.pt')
            if self.do_a2b_conf:
                torch.save({'a2b': self.gen_a2b_s[i].state_dict()}, gen_name.replace('gen_', 'a2b_gen_'))
                torch.save({'a2b': self.dis_a2b_s[i].state_dict()}, dis_name.replace('dis_', 'a2b_dis_'))
            if self.do_b2a_conf:
                torch.save({'b2a': self.gen_b2a_s[i].state_dict()}, gen_name.replace('gen_', 'b2a_gen_'))
                torch.save({'b2a': self.dis_b2a_s[i].state_dict()}, dis_name.replace('dis_', 'b2a_dis_'))
            if self.do_dis_council:
                if self.do_a2b_conf:
                    torch.save({'a2b': self.dis_council_a2b_s[i].state_dict()}, dis_council_name.replace('dis_council_', 'a2b_dis_council_'))
                if self.do_b2a_conf:
                    torch.save({'b2a': self.dis_council_b2a_s[i].state_dict()},  dis_council_name.replace('dis_council_', 'b2a_dis_council_'))
                torch.save({'gen': self.gen_opt_s[i].state_dict(), 'dis': self.dis_opt_s[i].state_dict(),
                            'dis_council': self.dis_council_opt_s[i].state_dict()}, opt_name)
            else:
                torch.save({'gen': self.gen_opt_s[i].state_dict(), 'dis': self.dis_opt_s[i].state_dict()}, opt_name)

#%%


#download.sh

In [6]:

#!/usr/bin/env bash
%%shell

FILE="pretrain_selfie_to_anime"
#pretrain_selfie_to_anime, pretrain_male_to_female, pretrain_glasses_removal

#celeba dataset
# @inproceedings{liu2015faceattributes,
# title = {Deep Learning Face Attributes in the Wild},
# author = {Liu, Ziwei and Luo, Ping and Wang, Xiaogang and Tang, Xiaoou},
# booktitle = {Proceedings of International Conference on Computer Vision (ICCV)},
# month = {December},
# year = {2015}
#}
if [ $FILE == "celeba_male2female" ]; then
    URL=https://cgm.technion.ac.il/Computer-Graphics-Multimedia/CouncilGAN/DataSet/celeba_male2female.zip?dl=0
    ZIP_FILE=./datasets/celeba_male2female.zip
    mkdir -p ./datasets/
    wget -N $URL -O $ZIP_FILE
    unzip $ZIP_FILE -d ./datasets/
    rm $ZIP_FILE

elif [ $FILE == "celeba_glasses_removal" ]; then
    URL=https://cgm.technion.ac.il/Computer-Graphics-Multimedia/CouncilGAN/DataSet/celeba_glasses.zip?dl=0
    ZIP_FILE=./datasets/celeba_glasses.zip
    mkdir -p ./datasets/
    wget -N $URL -O $ZIP_FILE
    unzip $ZIP_FILE -d ./datasets/
    rm $ZIP_FILE

#  selfie2anime dataset used in https://github.com/taki0112/UGATIT
#  title={U-GAT-IT: Unsupervised Generative Attentional Networks with Adaptive Layer-Instance Normalization for Image-to-Image Translation},
#  author={Kim, Junho and Kim, Minjae and Kang, Hyeonwoo and Lee, Kwanghee},
#  journal={arXiv preprint arXiv:1907.10830},
#  year={2019}
elif [ $FILE == "U_GAT_IT_selfie2anime" ]; then
  URL=https://www.dropbox.com/s/9lz6gwwwyyxpdnn/selfie2anime.zip?dl=0
  ZIP_FILE=./datasets/selfie2anime.zip
  mkdir -p ./datasets/
  wget -N $URL -O $ZIP_FILE
  unzip $ZIP_FILE -d ./datasets/
  rm $ZIP_FILE

elif [ $FILE == "pretrain_male_to_female" ]; then
    URL="https://onedrive.live.com/download?cid=552378614E6BA583&resid=552378614E6BA583%2119549&authkey=ANzlMZsBQy77urU"
    ZIP_FILE=./pretrain/pretrain_m2f.zip

    mkdir -p ./pretrain/
    wget -N $URL -O $ZIP_FILE
    unzip $ZIP_FILE -d ./pretrain/
    rm $ZIP_FILE

elif [ $FILE == "pretrain_glasses_removal" ]; then
    URL="https://onedrive.live.com/download?cid=552378614E6BA583&resid=552378614E6BA583%2119556&authkey=AAjSgID8ia6_Wt8"
    ZIP_FILE=./pretrain/pretrain_glasses_removal.zip
    mkdir -p ./pretrain/
    wget -N $URL -O $ZIP_FILE
    unzip $ZIP_FILE -d ./pretrain/
    rm $ZIP_FILE

elif [ $FILE == "pretrain_selfie_to_anime" ]; then
    URL="https://onedrive.live.com/download?cid=552378614E6BA583&resid=552378614E6BA583%2119566&authkey=AIcUuinR_eQPp1c"
    ZIP_FILE=./pretrain/pretrain_anime.zip
    mkdir -p ./pretrain/
    wget -N $URL -O $ZIP_FILE
    unzip $ZIP_FILE -d ./pretrain/
    rm $ZIP_FILE


else
    echo "Available arguments are celeba_male2female, celeba_glasses_removal, selfie2anime, pretrain_male_to_female, pretrain_glasses_removal, pretrain_selfie_to_anime."
    exit 1

fi


for details.

--2020-08-03 12:32:05--  https://onedrive.live.com/download?cid=552378614E6BA583&resid=552378614E6BA583%2119566&authkey=AIcUuinR_eQPp1c
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dy0vaw.db.files.1drv.com/y4mshr9NF5IshTjajuvYkWlN4LgRXxP6nyZT2CecF7IFpaCl8BEucQZPGhan16Uao6T5QdE_1jrMu8KOVLBX5w7XJE3j8Z5qvTaobrvcAO_6xFAI6hEWHNTB7RgqebO2VxJtxrd72I2Li3jBlbaDAcxAXCzwN7xibJnZf1g84Okvs0Il2WYhmSerr0UFeoFtszJ-Z222PB5ug50wi320zQryg/pretrain_anime.zip?download&psid=1 [following]
--2020-08-03 12:32:05--  https://dy0vaw.db.files.1drv.com/y4mshr9NF5IshTjajuvYkWlN4LgRXxP6nyZT2CecF7IFpaCl8BEucQZPGhan16Uao6T5QdE_1jrMu8KOVLBX5w7XJE3j8Z5qvTaobrvcAO_6xFAI6hEWHNTB7RgqebO2VxJtxrd72I2Li3jBlbaDAcxAXCzwN7xibJnZf1g84Okvs0Il2WYhmSerr0UFeoFtszJ-Z222PB5ug50wi320zQryg/pretrain_anime.zip?download&psid=1
Resolving dy0vaw.db.files.1drv.co

# test_on_folder.py


In [13]:
#@title input_configs { run: "auto" }
config = './pretrain/anime/256/anime2face_council_folder.yaml'  #@param {type: "string"}
checkpoint = './pretrain/anime/256/b2a_gen_0_01000000.pt'  #@param {type: "string"}
input_folder = './input_folder'  #@param {type: "string"}
output_folder = './output_folder'  #@param {type: "string"}
output_path = 'outputs' #@param {type: "string"}
a2b = 1 #@param {type: "integer"} #1 for a2b 0 for b2a
seed = 1 #@param {type: "integer"}
num_style = 10 #@param {type: "integer"}
output_only = False #@param{type: "boolean"}
num_of_images_to_test = 100 #@param {type: "integer"}
os.makedirs(input_folder, exist_ok=True)
print(f"Upload images into {input_folder}")
config = get_config(config)


Upload images into ./input_folder


# test_on_folder.py

In [14]:
from __future__ import print_function
# from utils import get_config, get_data_loader_folder, pytorch03_to_pytorch04
# from trainer_council import Council_Trainer
from torch import nn
from scipy.stats import entropy
import torch.nn.functional as F
import argparse
from torch.autograd import Variable
# from data import ImageFolder
import numpy as np
import torchvision.utils as vutils
try:
    from itertools import izip as zip
except ImportError: # will be 3.x series
    pass
import torch
import os
from tqdm import tqdm

data_name = 'out'

torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Load experiment setting
input_dim = config['input_dim_a'] if a2b else config['input_dim_b']
council_size = config['council']['council_size']


# Setup model and data loader
try:
    image_names = ImageFolder(input_folder, transform=None, return_paths=True)
except:
  print('Put images to be translated in ./input_folder')
  input("Press Enter when done")
  image_names = ImageFolder(input_folder, transform=None, return_paths=True)

if not 'new_size_a' in config.keys():
    config['new_size_a'] = config['new_size']
is_data_A = a2b
data_loader = get_data_loader_folder(input_folder, 1, False,\
                                     new_size=config['new_size_a'] if 'new_size_a' in config.keys() else config['new_size'],\
                                     crop=False, config=config, is_data_A=is_data_A)


style_dim = config['gen']['style_dim']
trainer = Council_Trainer(config)
only_one = False
if 'gen_' in checkpoint[-21:]:
    state_dict = torch.load(checkpoint, map_location='cuda:0')
    try:
        if a2b:
            trainer.gen_a2b_s[0].load_state_dict(state_dict['a2b'])
        else:
            trainer.gen_b2a_s[0].load_state_dict(state_dict['b2a'])
    except:
       print('opts.a2b should be set to ' + str(not a2b) + ' , Or config file could be wrong')
       a2b = not a2b
       if a2b:
           trainer.gen_a2b_s[0].load_state_dict(state_dict['a2b'])
       else:
           trainer.gen_b2a_s[0].load_state_dict(state_dict['b2a'])
            
    council_size = 1
    only_one = True
else:
    for i in range(council_size):
        try:
            if a2b:
                tmp_checkpoint = checkpoint[:-8] + 'a2b_gen_' + str(i) + '_' + opts.checkpoint[-8:] + '.pt'
                state_dict = torch.load(tmp_checkpoint)
                trainer.gen_a2b_s[i].load_state_dict(state_dict['a2b'])
            else:
                tmp_checkpoint = checkpoint[:-8] + 'b2a_gen_' + str(i) + '_' + checkpoint[-8:] + '.pt'
                state_dict = torch.load(tmp_checkpoint)
                trainer.gen_b2a_s[i].load_state_dict(state_dict['b2a'])
        except:
            print('opts.a2b should be set to ' + str(not a2b) + ' , Or config file could be wrong')
            
            opts.a2b = not a2b
            if a2b:
                tmp_checkpoint = checkpoint[:-8] + 'a2b_gen_' + str(i) + '_' + checkpoint[-8:] + '.pt'
                state_dict = torch.load(tmp_checkpoint)
                trainer.gen_a2b_s[i].load_state_dict(state_dict['a2b'])
            else:
                tmp_checkpoint = checkpoint[:-8] + 'b2a_gen_' + str(i) + '_' + checkpoint[-8:] + '.pt'
                state_dict = torch.load(tmp_checkpoint)
                trainer.gen_b2a_s[i].load_state_dict(state_dict['b2a'])
            


trainer.cuda()
trainer.eval()

encode_s = []
decode_s = []
if a2b:
    for i in range(council_size):
        encode_s.append(trainer.gen_a2b_s[i].encode)  # encode function
        decode_s.append(trainer.gen_a2b_s[i].decode)  # decode function
else:
    for i in range(council_size):
        encode_s.append(trainer.gen_b2a_s[i].encode)  # encode function
        decode_s.append(trainer.gen_b2a_s[i].decode)  # decode function


# creat testing images
num_of_images_to_test = num_of_images_to_test
seed = 1
curr_image_num = -1
for i, (images, names) in tqdm(enumerate(zip(data_loader, image_names)), total=num_of_images_to_test):
    if curr_image_num == num_of_images_to_test:
        break
    curr_image_num += 1
    k = np.random.randint(council_size)
    style_fixed = Variable(torch.randn(num_style, style_dim, 1, 1).cuda(), volatile=True)
    print(names[1])
    images = Variable(images.cuda(), volatile=True)

    content, _ = encode_s[k](images)
    seed += 1
    torch.random.manual_seed(seed)
    style = Variable(torch.randn(num_style, style_dim, 1, 1).cuda(), volatile=True)
    for j in range(num_style):
        s = style[j].unsqueeze(0)
        outputs = decode_s[k](content, s, images)
        basename = os.path.basename(names[1])
        output_folder = os.path.join(output_path, 'test_res')
        if only_one:
            path = os.path.join(output_folder, checkpoint[-11:-3] + "_%02d" % j, data_name + '_out_' + str(curr_image_num) + '_' + str(j) + '.jpg')
            path_all_in_one = os.path.join(output_folder, checkpoint[-11:-3] + '_all_in_1', data_name + '_out_' + str(curr_image_num) + '_' + str(j) + '.jpg')

        else:
            path = os.path.join(output_folder, checkpoint[-8:] + "_%02d" % j, data_name + '_out_' + str(curr_image_num) + '_' + str(j) + '.jpg')
            path_all_in_one = os.path.join(output_folder,  checkpoint[-8:] + '_all_in_1', data_name + '_out_' + str(curr_image_num) + '_' + str(j) + '.jpg')

        if not os.path.exists(os.path.dirname(path)):
            os.makedirs(os.path.dirname(path))
        vutils.save_image(outputs.data, path, padding=0, normalize=True)
        do_all_in_one = True
        if do_all_in_one:
            if not os.path.exists(os.path.dirname(path_all_in_one)):
                os.makedirs(os.path.dirname(path_all_in_one))
            vutils.save_image(outputs.data, path_all_in_one, padding=0, normalize=True)
    if not output_only:
        # also save input images
        output_folder = os.path.join(output_folder, 'input')
        if not os.path.exists(output_folder):
            os.makedirs(output_folder)
        vutils.save_image(images.data, os.path.join(output_folder, 'input{:03d}.jpg'.format(i)), padding=0, normalize=True)


opts.a2b should be set to False , Or config file could be wrong


  0%|          | 0/100 [00:00<?, ?it/s]

./input_folder/female_14491.jpg


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:114: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:116: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:121: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  1%|          | 1/100 [00:00<01:28,  1.12it/s]

./input_folder/female_14964.jpg


  2%|▏         | 2/100 [00:01<01:24,  1.16it/s]

./input_folder/female_15215.jpg


  3%|▎         | 3/100 [00:02<01:21,  1.19it/s]


TypeError: ignored